In [15]:
#GPU count and name
!nvidia-smi -L

GPU 0: GeForce RTX 2080 Ti (UUID: GPU-616398ce-ae25-c52b-7c47-f658d30d9d86)


In [ ]:
!pip install tensorflow-gpu==2.2.0

In [ ]:
!pip install Keras==2.3.1

In [18]:
!python -c 'import keras; print(keras.__version__)'

Using TensorFlow backend.
2.3.1


## Set paths

In [2]:
## Set paths
import os
DATASET_BASE_DIR = '../datasets/data'
DATASET_VERSION = 'v5'
DATASET_VERSION_DIR = os.path.join(DATASET_BASE_DIR, DATASET_VERSION)
CLASSES_FILE = os.path.join(DATASET_VERSION_DIR, 'classes.csv')
TRAIN_ANNOTATIONS = os.path.join(DATASET_VERSION_DIR, 'train_iter3.csv')
VAL_ANNOTATIONS = os.path.join(DATASET_VERSION_DIR, 'val_iter3.csv')

TRAINING_BASE_DIR = os.path.join('../trainings', 'v6')

## Check directories and files

In [3]:
!wc -l {TRAIN_ANNOTATIONS}

6412 ../datasets/data/v5/train_iter3.csv


In [4]:
!wc -l {VAL_ANNOTATIONS}

789 ../datasets/data/v5/val_iter3.csv


In [5]:
!wc -l {CLASSES_FILE}

19 ../datasets/data/v5/classes.csv


## Set pretrained model

In [6]:
# utilize best weights from 4 classes model as baseline
PRETRAINED_MODEL = '/home/aikauel/enap/aerialnet_project/trainings/v5/snapshots/resnet50_csv_228.h5'

# Compute best anchors for dataset annotations

In [7]:
!anchor-optimization {TRAIN_ANNOTATIONS} --no-resize --ratios=3 --threads 100

Using TensorFlow backend.
Optimization ended successfully!

Final best anchor configuration
State: 0.17071
Ratios: [0.515, 1.0, 1.942]
Scales: [0.446, 0.716, 1.132]
Number of labels that don't have any matching anchor: 52


Save anchor optimization to ...trainings/vx/config.ini file

# Set number of steps per epoch

In [8]:
import pandas as pd
import math

BATCH_SIZE = 8

# compute number of annotations to compute number of steps per epoch
df = pd.read_csv(TRAIN_ANNOTATIONS, header=None, names=['img_path', 'x1', 'y1', 'x2', 'y2', 'class'])

countAnn = len(df)-1
countImg = df['img_path'].nunique()
no_steps = math.ceil(countImg/BATCH_SIZE)

print("Count of images: {}".format(countImg))
print("Count of annotations: {}".format(countAnn))
print("Number of steps per epoch: {}".format(no_steps))

Count of images: 4035
Count of annotations: 6412
Number of steps per epoch: 505


# Train model

In [9]:
CONFIG_FILE = os.path.join(TRAINING_BASE_DIR, 'config.ini')
TENSORBOARD_LOGS_DIR = os.path.join(TRAINING_BASE_DIR, 'logs')
SNAPSHOTS_DIR = os.path.join(TRAINING_BASE_DIR, 'snapshots')

In [10]:
TENSORBOARD_LOGS_DIR

'../trainings/v6/logs'

### FCNs training (regression, classification) with freezed backbone  

First, train the model freezing the backbone. The backbone weights will be the weights from the previous model train before.
Set gamma with high value (>2.0) to weight the loss of well-classified classes down, forcing the model to learn on harder (in this case: less examples) classes.

In [11]:
!python ../../keras-retinanet/keras_retinanet/bin/train.py \
--config {CONFIG_FILE} \
--freeze-backbone \
--random-transform \
--weights {PRETRAINED_MODEL} \
--weighted-average \
--batch-size {BATCH_SIZE} \
--steps {no_steps} \
--epochs 100 \
--no-resize \
--image-min-side 1000 \
--image-max-side 1000 \
--compute-val-loss \
--multiprocessing \
--workers 2 \
--tensorboard-dir {TENSORBOARD_LOGS_DIR} \
--snapshot-path {SNAPSHOTS_DIR} \
--validation-freq 5 \
--class-names '/home/aikauel/enap/aerialnet_project/datasets/data/v5/CLASSES_REPORT_NAMES' \
csv {TRAIN_ANNOTATIONS} {CLASSES_FILE} \
--val-annotations {VAL_ANNOTATIONS}

Using TensorFlow backend.
classNames: ['Animal', 'Basural-Escombro-MConstrucción', 'Bus', 'Camión', 'Chasis', 'Cilindro', 'Estructura', 'GHorquilla', 'Juegos', 'Maquinaria', 'PalletCaja', 'Persona', 'Pickup', 'Piscina', 'Poste', 'SAdvertencia', 'Tractor', 'Troncos', 'Tuberia', 'Vehículo']
Creating model, this may take a second...
2020-11-11 17:42:06.073601: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-11-11 17:42:06.103975: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-11-11 17:42:06.104274: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.755GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2020-11-11 17:42:06.1

tracking <tf.Variable 'Variable:0' shape=(9, 4) dtype=float32, numpy=
array([[ -39.775097,  -20.484173,   39.775097,   20.484173],
       [ -63.85419 ,  -32.884907,   63.85419 ,   32.884907],
       [-100.95383 ,  -51.99122 ,  100.95383 ,   51.99122 ],
       [ -28.544   ,  -28.544   ,   28.544   ,   28.544   ],
       [ -45.824   ,  -45.824   ,   45.824   ,   45.824   ],
       [ -72.448   ,  -72.448   ,   72.448   ,   72.448   ],
       [ -20.482843,  -39.77768 ,   20.482843,   39.77768 ],
       [ -32.88277 ,  -63.85834 ,   32.88277 ,   63.85834 ],
       [ -51.98784 , -100.96039 ,   51.98784 ,  100.96039 ]],
      dtype=float32)> anchors
tracking <tf.Variable 'Variable:0' shape=(9, 4) dtype=float32, numpy=
array([[ -79.55019 ,  -40.968346,   79.55019 ,   40.968346],
       [-127.70838 ,  -65.76981 ,  127.70838 ,   65.76981 ],
       [-201.90765 , -103.98244 ,  201.90765 ,  103.98244 ],
       [ -57.088   ,  -57.088   ,   57.088   ,   57.088   ],
       [ -91.648   ,  -91.648   ,   

res3d_branch2a (Conv2D)         (None, None, None, 1 65536       res3c_relu[0][0]                 
__________________________________________________________________________________________________
bn3d_branch2a (BatchNormalizati (None, None, None, 1 512         res3d_branch2a[0][0]             
__________________________________________________________________________________________________
res3d_branch2a_relu (Activation (None, None, None, 1 0           bn3d_branch2a[0][0]              
__________________________________________________________________________________________________
padding3d_branch2b (ZeroPadding (None, None, None, 1 0           res3d_branch2a_relu[0][0]        
__________________________________________________________________________________________________
res3d_branch2b (Conv2D)         (None, None, None, 1 147456      padding3d_branch2b[0][0]         
__________________________________________________________________________________________________
bn3d_branc

2020-11-11 17:42:07.866612: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1408] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI_ERROR_INSUFFICIENT_PRIVILEGES
2020-11-11 17:42:07.868176: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1447] function cupti_interface_->ActivityRegisterCallbacks( AllocCuptiActivityBuffer, FreeCuptiActivityBuffer)failed with error CUPTI_ERROR_INSUFFICIENT_PRIVILEGES
2020-11-11 17:42:07.868226: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1430] function cupti_interface_->EnableCallback( 0 , subscriber_, CUPTI_CB_DOMAIN_DRIVER_API, cbid)failed with error CUPTI_ERROR_INVALID_PARAMETER
Epoch 1/100
2020-11-11 17:42:15.687636: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2020-11-11 17:42:16.949866: W tensorflow/stream_executor/gpu/asm_compiler.cc:81] Running ptxas --version returned 256
2020-11-11 17:4

Epoch 3/100
505/505 [==============================] - 343s 680ms/step - loss: 0.2989 - regression_loss: 0.2020 - classification_loss: 0.0969
Running network: 100% (524 of 524) |#####| Elapsed Time: 0:00:45 Time:  0:00:45
Parsing annotations: 100% (524 of 524) |#| Elapsed Time: 0:00:00 Time:  0:00:00
99 instances of class Animal with average precision: 0.6374 TP: 80 FP: 1200 precision: 0.0625 recall: 0.8081 f1_score: 0.1160 f2_score: 0.2387
65 instances of class Basural-Escombro-MConstrucción with average precision: 0.1409 TP: 37 FP: 1089 precision: 0.0329 recall: 0.5692 f1_score: 0.0621 f2_score: 0.1335
0 instances of class Bus with average precision: 0.0000 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_score: 0.0000 f2_score: 0.0000
139 instances of class Camión with average precision: 0.9850 TP: 139 FP: 430 precision: 0.2443 recall: 1.0000 f1_score: 0.3927 f2_score: 0.6178
69 instances of class Chasis with average precision: 0.9750 TP: 69 FP: 493 precision: 0.1228 recall: 1.0000 f

...

Epoch 18/100
505/505 [==============================] - 345s 683ms/step - loss: 0.1819 - regression_loss: 0.1367 - classification_loss: 0.0453 - val_loss: 1.8695 - val_regression_loss: 0.2450 - val_classification_loss: 0.1464
Running network: 100% (524 of 524) |#####| Elapsed Time: 0:00:44 Time:  0:00:44
Parsing annotations: 100% (524 of 524) |#| Elapsed Time: 0:00:00 Time:  0:00:00
99 instances of class Animal with average precision: 0.7511 TP: 88 FP: 412 precision: 0.1760 recall: 0.8889 f1_score: 0.2938 f2_score: 0.4911
65 instances of class Basural-Escombro-MConstrucción with average precision: 0.2162 TP: 40 FP: 779 precision: 0.0488 recall: 0.6154 f1_score: 0.0905 f2_score: 0.1854
0 instances of class Bus with average precision: 0.0000 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_score: 0.0000 f2_score: 0.0000
139 instances of class Camión with average precision: 0.9844 TP: 138 FP: 134 precision: 0.5074 recall: 0.9928 f1_score: 0.6715 f2_score: 0.8333
69 instances of class Chasi

Epoch 21/100
505/505 [==============================] - 341s 676ms/step - loss: 0.1817 - regression_loss: 0.1367 - classification_loss: 0.0450 - val_loss: 1.8621 - val_regression_loss: 0.2453 - val_classification_loss: 0.1468
Running network: 100% (524 of 524) |#####| Elapsed Time: 0:00:44 Time:  0:00:44
Parsing annotations: 100% (524 of 524) |#| Elapsed Time: 0:00:00 Time:  0:00:00
99 instances of class Animal with average precision: 0.7397 TP: 88 FP: 427 precision: 0.1709 recall: 0.8889 f1_score: 0.2866 f2_score: 0.4830
65 instances of class Basural-Escombro-MConstrucción with average precision: 0.2172 TP: 40 FP: 767 precision: 0.0496 recall: 0.6154 f1_score: 0.0917 f2_score: 0.1874
0 instances of class Bus with average precision: 0.0000 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_score: 0.0000 f2_score: 0.0000
139 instances of class Camión with average precision: 0.9846 TP: 138 FP: 123 precision: 0.5287 recall: 0.9928 f1_score: 0.6900 f2_score: 0.8446
69 instances of class Chasi


Epoch 00023: ReduceLROnPlateau reducing learning rate to 1.0000000116860975e-08.
Epoch 24/100
505/505 [==============================] - 343s 679ms/step - loss: 0.1817 - regression_loss: 0.1368 - classification_loss: 0.0449 - val_loss: 1.8621 - val_regression_loss: 0.2453 - val_classification_loss: 0.1468
Running network: 100% (524 of 524) |#####| Elapsed Time: 0:00:44 Time:  0:00:44
Parsing annotations: 100% (524 of 524) |#| Elapsed Time: 0:00:00 Time:  0:00:00
99 instances of class Animal with average precision: 0.7423 TP: 88 FP: 415 precision: 0.1750 recall: 0.8889 f1_score: 0.2924 f2_score: 0.4894
65 instances of class Basural-Escombro-MConstrucción with average precision: 0.2164 TP: 40 FP: 783 precision: 0.0486 recall: 0.6154 f1_score: 0.0901 f2_score: 0.1847
0 instances of class Bus with average precision: 0.0000 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_score: 0.0000 f2_score: 0.0000
139 instances of class Camión with average precision: 0.9844 TP: 138 FP: 130 precision: 0

Epoch 27/100
505/505 [==============================] - 344s 681ms/step - loss: 0.1809 - regression_loss: 0.1356 - classification_loss: 0.0453 - val_loss: 1.8565 - val_regression_loss: 0.2431 - val_classification_loss: 0.1488
Running network: 100% (524 of 524) |#####| Elapsed Time: 0:00:44 Time:  0:00:44
Parsing annotations: 100% (524 of 524) |#| Elapsed Time: 0:00:00 Time:  0:00:00
99 instances of class Animal with average precision: 0.7425 TP: 88 FP: 416 precision: 0.1746 recall: 0.8889 f1_score: 0.2919 f2_score: 0.4889
65 instances of class Basural-Escombro-MConstrucción with average precision: 0.2163 TP: 40 FP: 783 precision: 0.0486 recall: 0.6154 f1_score: 0.0901 f2_score: 0.1847
0 instances of class Bus with average precision: 0.0000 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_score: 0.0000 f2_score: 0.0000
139 instances of class Camión with average precision: 0.9844 TP: 138 FP: 131 precision: 0.5130 recall: 0.9928 f1_score: 0.6765 f2_score: 0.8364
69 instances of class Chasi

Epoch 30/100
505/505 [==============================] - 377s 746ms/step - loss: 0.1782 - regression_loss: 0.1331 - classification_loss: 0.0451 - val_loss: 1.8565 - val_regression_loss: 0.2430 - val_classification_loss: 0.1488
Running network: 100% (524 of 524) |#####| Elapsed Time: 0:00:44 Time:  0:00:44
Parsing annotations: 100% (524 of 524) |#| Elapsed Time: 0:00:00 Time:  0:00:00
99 instances of class Animal with average precision: 0.7425 TP: 88 FP: 416 precision: 0.1746 recall: 0.8889 f1_score: 0.2919 f2_score: 0.4889
65 instances of class Basural-Escombro-MConstrucción with average precision: 0.2163 TP: 40 FP: 783 precision: 0.0486 recall: 0.6154 f1_score: 0.0901 f2_score: 0.1847
0 instances of class Bus with average precision: 0.0000 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_score: 0.0000 f2_score: 0.0000
139 instances of class Camión with average precision: 0.9844 TP: 138 FP: 131 precision: 0.5130 recall: 0.9928 f1_score: 0.6765 f2_score: 0.8364
69 instances of class Chasi

#### Full model training

With a decent mAP result in previous step, we now unfreeze the backbone weights to train the full model.

In [34]:
BATCH_SIZE = 4

# compute number of annotations to compute number of steps per epoch
df = pd.read_csv(TRAIN_ANNOTATIONS, header=None, names=['img_path', 'x1', 'y1', 'x2', 'y2', 'class'])

countAnn = len(df)-1
countImg = df['img_path'].nunique()
no_steps = math.ceil(countImg/BATCH_SIZE)

print("Count of images: {}".format(countImg))
print("Count of annotations: {}".format(countAnn))
print("Number of steps per epoch: {}".format(no_steps))

Count of images: 496
Count of annotations: 2873
Number of steps per epoch: 124


In [36]:
last_model = os.path.join(SNAPSHOTS_DIR, 'resnet50_csv_36.h5')

!python ../../keras-retinanet/keras_retinanet/bin/train.py \
--config {CONFIG_FILE} \
--random-transform \
--weights {last_model} \
--initial-epoch 36 \
--weighted-average \
--batch-size {BATCH_SIZE} \
--steps {no_steps} \
--epochs 100 \
--no-resize \
--image-min-side 1000 \
--image-max-side 1000 \
--compute-val-loss \
--multiprocessing \
--workers 2 \
--tensorboard-dir {TENSORBOARD_LOGS_DIR} \
--snapshot-path {SNAPSHOTS_DIR} \
--validation-freq 5 \
--lr 1e-5 \
--reduce-lr-patience 5 \
--reduce-lr-factor 0.1 \
--gamma 3.0 \
--class-names '/home/aikauel/enap/aerialnet_project/datasets/data/v5/CLASSES_REPORT_NAMES' \
csv {TRAIN_ANNOTATIONS} {CLASSES_FILE} \
--val-annotations {VAL_ANNOTATIONS}

Using TensorFlow backend.
classNames: ['Animal', 'Basural-Escombro-MConstrucción', 'Bus', 'Camión', 'Chasis', 'Cilindro', 'Estructura', 'GHorquilla', 'Juegos', 'Maquinaria', 'PalletCaja', 'Persona', 'Pickup', 'Piscina', 'Poste', 'SAdvertencia', 'Tractor', 'Troncos', 'Tuberia', 'Vehículo']
Creating model, this may take a second...
2020-11-04 07:11:22.303646: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-11-04 07:11:22.334952: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-11-04 07:11:22.335284: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.755GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2020-11-04 07:11:22.3

tracking <tf.Variable 'Variable:0' shape=(27, 4) dtype=float32, numpy=
array([[ -57.984   ,  -14.496   ,   57.984   ,   14.496   ],
       [ -92.032   ,  -23.008   ,   92.032   ,   23.008   ],
       [-146.432   ,  -36.608   ,  146.432   ,   36.608   ],
       [ -45.556534,  -18.450396,   45.556534,   18.450396],
       [ -72.30717 ,  -29.284403,   72.30717 ,   29.284403],
       [-115.04785 ,  -46.59438 ,  115.04785 ,   46.59438 ],
       [ -38.707645,  -21.714989,   38.707645,   21.714989],
       [ -61.436634,  -34.465954,   61.436634,   34.465954],
       [ -97.75176 ,  -54.838737,   97.75176 ,   54.838737],
       [ -33.432533,  -25.141264,   33.432533,   25.141264],
       [ -53.063995,  -39.90412 ,   53.063995,   39.90412 ],
       [ -84.43006 ,  -63.491405,   84.43006 ,   63.491405],
       [ -28.992   ,  -28.992   ,   28.992   ,   28.992   ],
       [ -46.016   ,  -46.016   ,   46.016   ,   46.016   ],
       [ -73.216   ,  -73.216   ,   73.216   ,   73.216   ],
       [ -25.1

2020-11-04 07:11:24.428861: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1408] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI_ERROR_INSUFFICIENT_PRIVILEGES
2020-11-04 07:11:24.433438: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1447] function cupti_interface_->ActivityRegisterCallbacks( AllocCuptiActivityBuffer, FreeCuptiActivityBuffer)failed with error CUPTI_ERROR_INSUFFICIENT_PRIVILEGES
2020-11-04 07:11:24.433555: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1430] function cupti_interface_->EnableCallback( 0 , subscriber_, CUPTI_CB_DOMAIN_DRIVER_API, cbid)failed with error CUPTI_ERROR_INVALID_PARAMETER
Epoch 37/100
2020-11-04 07:11:39.905457: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2020-11-04 07:11:40.813616: W tensorflow/stream_executor/gpu/asm_compiler.cc:81] Running ptxas --version returned 256
2020-11-04 07:

Epoch 38/100
124/124 [==============================] - 85s 683ms/step - loss: 1.5838 - regression_loss: 1.3289 - classification_loss: 0.2548
Running network: 100% (45 of 45) |#######| Elapsed Time: 0:00:06 Time:  0:00:06
Parsing annotations: 100% (45 of 45) |###| Elapsed Time: 0:00:00 Time:  0:00:00
99 instances of class Animal with average precision: 0.6753 TP: 86 FP: 352 precision: 0.1963 recall: 0.8687 f1_score: 0.3203 f2_score: 0.5156
65 instances of class Basural-Escombro-MConstrucción with average precision: 0.1597 TP: 40 FP: 898 precision: 0.0426 recall: 0.6154 f1_score: 0.0798 f2_score: 0.1669
0 instances of class Bus with average precision: 0.0000 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_score: 0.0000 f2_score: 0.0000
0 instances of class Camión with average precision: 0.0000 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_score: 0.0000 f2_score: 0.0000
0 instances of class Chasis with average precision: 0.0000 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_score: 0.0

Epoch 41/100
124/124 [==============================] - 85s 683ms/step - loss: 1.5190 - regression_loss: 1.2744 - classification_loss: 0.2447 - val_loss: 2.3000 - val_regression_loss: 1.7631 - val_classification_loss: 0.3572
Running network: 100% (45 of 45) |#######| Elapsed Time: 0:00:06 Time:  0:00:06
Parsing annotations: 100% (45 of 45) |###| Elapsed Time: 0:00:00 Time:  0:00:00
99 instances of class Animal with average precision: 0.7111 TP: 89 FP: 304 precision: 0.2265 recall: 0.8990 f1_score: 0.3618 f2_score: 0.5640
65 instances of class Basural-Escombro-MConstrucción with average precision: 0.1946 TP: 46 FP: 1090 precision: 0.0405 recall: 0.7077 f1_score: 0.0766 f2_score: 0.1648
0 instances of class Bus with average precision: 0.0000 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_score: 0.0000 f2_score: 0.0000
0 instances of class Camión with average precision: 0.0000 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_score: 0.0000 f2_score: 0.0000
0 instances of class Chasis with 

...

Epoch 86/100
124/124 [==============================] - 87s 703ms/step - loss: 1.1438 - regression_loss: 0.9734 - classification_loss: 0.1704 - val_loss: 2.0734 - val_regression_loss: 1.6506 - val_classification_loss: 0.3411
Running network: 100% (45 of 45) |#######| Elapsed Time: 0:00:05 Time:  0:00:05
Parsing annotations: 100% (45 of 45) |###| Elapsed Time: 0:00:00 Time:  0:00:00
99 instances of class Animal with average precision: 0.7460 TP: 88 FP: 290 precision: 0.2328 recall: 0.8889 f1_score: 0.3690 f2_score: 0.5685
65 instances of class Basural-Escombro-MConstrucción with average precision: 0.2174 TP: 46 FP: 801 precision: 0.0543 recall: 0.7077 f1_score: 0.1009 f2_score: 0.2078
0 instances of class Bus with average precision: 0.0000 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_score: 0.0000 f2_score: 0.0000
0 instances of class Camión with average precision: 0.0000 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_score: 0.0000 f2_score: 0.0000
0 instances of class Chasis with a

Epoch 89/100
124/124 [==============================] - 87s 705ms/step - loss: 1.1327 - regression_loss: 0.9628 - classification_loss: 0.1699 - val_loss: 2.0734 - val_regression_loss: 1.6506 - val_classification_loss: 0.3411
Running network: 100% (45 of 45) |#######| Elapsed Time: 0:00:05 Time:  0:00:05
Parsing annotations: 100% (45 of 45) |###| Elapsed Time: 0:00:00 Time:  0:00:00
99 instances of class Animal with average precision: 0.7387 TP: 86 FP: 373 precision: 0.1874 recall: 0.8687 f1_score: 0.3082 f2_score: 0.5029
65 instances of class Basural-Escombro-MConstrucción with average precision: 0.1814 TP: 44 FP: 817 precision: 0.0511 recall: 0.6769 f1_score: 0.0950 f2_score: 0.1963
0 instances of class Bus with average precision: 0.0000 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_score: 0.0000 f2_score: 0.0000
0 instances of class Camión with average precision: 0.0000 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_score: 0.0000 f2_score: 0.0000
0 instances of class Chasis with a

Epoch 92/100
124/124 [==============================] - 87s 703ms/step - loss: 1.1299 - regression_loss: 0.9627 - classification_loss: 0.1672 - val_loss: 2.1665 - val_regression_loss: 1.7299 - val_classification_loss: 0.3540
Running network: 100% (45 of 45) |#######| Elapsed Time: 0:00:06 Time:  0:00:06
Parsing annotations: 100% (45 of 45) |###| Elapsed Time: 0:00:00 Time:  0:00:00
99 instances of class Animal with average precision: 0.7610 TP: 88 FP: 344 precision: 0.2037 recall: 0.8889 f1_score: 0.3315 f2_score: 0.5314
65 instances of class Basural-Escombro-MConstrucción with average precision: 0.1709 TP: 46 FP: 790 precision: 0.0550 recall: 0.7077 f1_score: 0.1021 f2_score: 0.2099
0 instances of class Bus with average precision: 0.0000 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_score: 0.0000 f2_score: 0.0000
0 instances of class Camión with average precision: 0.0000 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_score: 0.0000 f2_score: 0.0000
0 instances of class Chasis with a

Epoch 95/100
124/124 [==============================] - 93s 750ms/step - loss: 1.1030 - regression_loss: 0.9378 - classification_loss: 0.1652 - val_loss: 2.1022 - val_regression_loss: 1.6768 - val_classification_loss: 0.3450
Running network: 100% (45 of 45) |#######| Elapsed Time: 0:00:06 Time:  0:00:06
Parsing annotations: 100% (45 of 45) |###| Elapsed Time: 0:00:00 Time:  0:00:00
99 instances of class Animal with average precision: 0.7518 TP: 88 FP: 353 precision: 0.1995 recall: 0.8889 f1_score: 0.3259 f2_score: 0.5257
65 instances of class Basural-Escombro-MConstrucción with average precision: 0.1842 TP: 44 FP: 684 precision: 0.0604 recall: 0.6769 f1_score: 0.1110 f2_score: 0.2227
0 instances of class Bus with average precision: 0.0000 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_score: 0.0000 f2_score: 0.0000
0 instances of class Camión with average precision: 0.0000 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_score: 0.0000 f2_score: 0.0000
0 instances of class Chasis with a

Epoch 98/100
124/124 [==============================] - 88s 706ms/step - loss: 1.0831 - regression_loss: 0.9261 - classification_loss: 0.1570 - val_loss: 2.1022 - val_regression_loss: 1.6768 - val_classification_loss: 0.3450
Running network: 100% (45 of 45) |#######| Elapsed Time: 0:00:06 Time:  0:00:06
Parsing annotations: 100% (45 of 45) |###| Elapsed Time: 0:00:00 Time:  0:00:00
99 instances of class Animal with average precision: 0.7396 TP: 87 FP: 366 precision: 0.1921 recall: 0.8788 f1_score: 0.3152 f2_score: 0.5124
65 instances of class Basural-Escombro-MConstrucción with average precision: 0.2168 TP: 42 FP: 593 precision: 0.0661 recall: 0.6462 f1_score: 0.1200 f2_score: 0.2346
0 instances of class Bus with average precision: 0.0000 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_score: 0.0000 f2_score: 0.0000
0 instances of class Camión with average precision: 0.0000 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_score: 0.0000 f2_score: 0.0000
0 instances of class Chasis with a

## Test model
#### Se procesan las imágenes del dataset de testing

In [12]:
# Convert training model to inference model
# path to trained weights
epoch = 21
last_model = os.path.join(SNAPSHOTS_DIR, 'resnet50_csv_{}.h5'.format(epoch))
inference_model = last_model.replace('snapshots', 'inference')

# convert trained model to inference model to be used for generating predictions
inference_model = last_model.replace('snapshots/', 'inference/')
!python ../../keras-retinanet/keras_retinanet/bin/convert_model.py {last_model} {inference_model} \
--config {CONFIG_FILE}

Using TensorFlow backend.
2020-11-11 21:18:42.650352: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-11-11 21:18:42.679316: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-11-11 21:18:42.679680: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.755GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2020-11-11 21:18:42.679818: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-11-11 21:18:42.684783: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2020-11-11 21:18:42.686895: I tensorf

#### Set paths

In [13]:
test_images_path = '/home/aikauel/enap/aerialnet_project/datasets/official_37_test_images_ori/'
#test_images_path = '/home/aikauel/enap/data/vitto/'
test_output_path = os.path.join(TRAINING_BASE_DIR, 
                                'official_37_output_epoch{}_threshold53'.format(epoch))

In [14]:
test_output_path

'../trainings/v6/official_37_output_epoch21_threshold53'

In [17]:
!python ../../aerialnet_project/shared_utils/image_inference_write.py \
--input_dir {test_images_path} \
--model {inference_model} \
--output_dir {test_output_path} \
--labels {CLASSES_FILE} \
--threshold 0.53 \
--no_csv True

Using TensorFlow backend.
/home/aikauel/enap/aerialnet_project/datasets/official_37_test_images_ori/
LABELS: {0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9', 10: '10', 11: '11', 12: '12', 13: '13', 14: '14', 15: '15', 16: '16', 17: '17', 18: '18', 19: '19'}
Output dir already exist
2020-11-11 21:23:47.062022: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-11-11 21:23:47.092107: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-11-11 21:23:47.092419: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.755GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2020-11-11 21:23:47.092604: I tensorfl

/home/aikauel/anaconda3/envs/enap/lib/python3.6/site-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
no csv: /home/aikauel/enap/aerialnet_project/datasets/official_37_test_images_ori/
[INFO] predicting image 1 of 37: 2020-08-11_08:13:26-3872762-DJI_0282.jpg
2020-11-11 21:23:50.368301: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2020-11-11 21:23:51.016945: W tensorflow/stream_executor/gpu/asm_compiler.cc:81] Running ptxas --version returned 256
2020-11-11 21:23:51.066392: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2020-11-11 21:23:52.222925: I tensorflow/

In [16]:
CLASSES_FILE

'../datasets/data/v5/classes.csv'

## Resultados de testing:
 #### Maquinaria: FP=1, TP=5, FN=5

# Iteración 2:
    # Se incluyen ejemplos del dataset de SkyNet

In [1]:
## Set paths
import os
DATASET_BASE_DIR = '../datasets/data'
DATASET_VERSION = 'v5'
DATASET_VERSION_DIR = os.path.join(DATASET_BASE_DIR, DATASET_VERSION)
CLASSES_FILE = os.path.join(DATASET_VERSION_DIR, 'classes.csv')
TRAIN_ANNOTATIONS = os.path.join(DATASET_VERSION_DIR, 'train_iter2.csv')
VAL_ANNOTATIONS = os.path.join(DATASET_VERSION_DIR, 'val_iter2.csv')

TRAINING_BASE_DIR = os.path.join('../trainings', DATASET_VERSION)

## Check directories and files

In [4]:
!wc -l {TRAIN_ANNOTATIONS}

4960 ../datasets/data/v5/train_iter2.csv


In [5]:
!wc -l {VAL_ANNOTATIONS}

584 ../datasets/data/v5/val_iter2.csv


In [6]:
!wc -l {CLASSES_FILE}

19 ../datasets/data/v5/classes.csv


# Set number of steps per epoch

# Train model

In [10]:
CONFIG_FILE = os.path.join(TRAINING_BASE_DIR, 'config.ini')
TENSORBOARD_LOGS_DIR = os.path.join(TRAINING_BASE_DIR, 'logs')
SNAPSHOTS_DIR = os.path.join(TRAINING_BASE_DIR, 'snapshots')

#### Full model training

In [11]:
BATCH_SIZE = 4

# compute number of annotations to compute number of steps per epoch
df = pd.read_csv(TRAIN_ANNOTATIONS, header=None, names=['img_path', 'x1', 'y1', 'x2', 'y2', 'class'])

countAnn = len(df)-1
countImg = df['img_path'].nunique()
no_steps = math.ceil(countImg/BATCH_SIZE)

print("Count of images: {}".format(countImg))
print("Count of annotations: {}".format(countAnn))
print("Number of steps per epoch: {}".format(no_steps))

Count of images: 2583
Count of annotations: 4960
Number of steps per epoch: 646


In [14]:
epoch = 97
last_model = os.path.join(SNAPSHOTS_DIR, 'resnet50_csv_{}.h5'.format(epoch))

!python ../../keras-retinanet/keras_retinanet/bin/train.py \
--config {CONFIG_FILE} \
--random-transform \
--weights {last_model} \
--initial-epoch {epoch} \
--weighted-average \
--batch-size {BATCH_SIZE} \
--steps {no_steps} \
--epochs 100 \
--no-resize \
--image-min-side 1000 \
--image-max-side 1000 \
--compute-val-loss \
--multiprocessing \
--workers 2 \
--tensorboard-dir {TENSORBOARD_LOGS_DIR} \
--snapshot-path {SNAPSHOTS_DIR} \
--validation-freq 5 \
--lr 1e-5 \
--reduce-lr-patience 5 \
--reduce-lr-factor 0.1 \
--gamma 3.0 \
--class-names '/home/aikauel/enap/aerialnet_project/datasets/data/v5/CLASSES_REPORT_NAMES' \
csv {TRAIN_ANNOTATIONS} {CLASSES_FILE} \
--val-annotations {VAL_ANNOTATIONS}

Using TensorFlow backend.
classNames: ['Animal', 'Basural-Escombro-MConstrucción', 'Bus', 'Camión', 'Chasis', 'Cilindro', 'Estructura', 'GHorquilla', 'Juegos', 'Maquinaria', 'PalletCaja', 'Persona', 'Pickup', 'Piscina', 'Poste', 'SAdvertencia', 'Tractor', 'Troncos', 'Tuberia', 'Vehículo']
Creating model, this may take a second...
2020-11-04 23:49:19.232472: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-11-04 23:49:19.262849: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-11-04 23:49:19.263179: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.755GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2020-11-04 23:49:19.2

tracking <tf.Variable 'Variable:0' shape=(27, 4) dtype=float32, numpy=
array([[ -57.984   ,  -14.496   ,   57.984   ,   14.496   ],
       [ -92.032   ,  -23.008   ,   92.032   ,   23.008   ],
       [-146.432   ,  -36.608   ,  146.432   ,   36.608   ],
       [ -45.556534,  -18.450396,   45.556534,   18.450396],
       [ -72.30717 ,  -29.284403,   72.30717 ,   29.284403],
       [-115.04785 ,  -46.59438 ,  115.04785 ,   46.59438 ],
       [ -38.707645,  -21.714989,   38.707645,   21.714989],
       [ -61.436634,  -34.465954,   61.436634,   34.465954],
       [ -97.75176 ,  -54.838737,   97.75176 ,   54.838737],
       [ -33.432533,  -25.141264,   33.432533,   25.141264],
       [ -53.063995,  -39.90412 ,   53.063995,   39.90412 ],
       [ -84.43006 ,  -63.491405,   84.43006 ,   63.491405],
       [ -28.992   ,  -28.992   ,   28.992   ,   28.992   ],
       [ -46.016   ,  -46.016   ,   46.016   ,   46.016   ],
       [ -73.216   ,  -73.216   ,   73.216   ,   73.216   ],
       [ -25.1

2020-11-04 23:49:21.057708: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1408] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI_ERROR_INSUFFICIENT_PRIVILEGES
2020-11-04 23:49:21.059503: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1447] function cupti_interface_->ActivityRegisterCallbacks( AllocCuptiActivityBuffer, FreeCuptiActivityBuffer)failed with error CUPTI_ERROR_INSUFFICIENT_PRIVILEGES
2020-11-04 23:49:21.059561: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1430] function cupti_interface_->EnableCallback( 0 , subscriber_, CUPTI_CB_DOMAIN_DRIVER_API, cbid)failed with error CUPTI_ERROR_INVALID_PARAMETER
Epoch 98/100
2020-11-04 23:49:36.870807: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2020-11-04 23:49:37.750277: W tensorflow/stream_executor/gpu/asm_compiler.cc:81] Running ptxas --version returned 256
2020-11-04 23:

Epoch 99/100
646/646 [==============================] - 442s 684ms/step - loss: 0.4744 - regression_loss: 0.3657 - classification_loss: 0.1087
Running network: 100% (319 of 319) |#####| Elapsed Time: 0:00:30 Time:  0:00:30
Parsing annotations: 100% (319 of 319) |#| Elapsed Time: 0:00:00 Time:  0:00:00
99 instances of class Animal with average precision: 0.7409 TP: 87 FP: 1426 precision: 0.0575 recall: 0.8788 f1_score: 0.1079 f2_score: 0.2279
65 instances of class Basural-Escombro-MConstrucción with average precision: 0.1981 TP: 45 FP: 1405 precision: 0.0310 recall: 0.6923 f1_score: 0.0594 f2_score: 0.1316
0 instances of class Bus with average precision: 0.0000 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_score: 0.0000 f2_score: 0.0000
88 instances of class Camión with average precision: 0.8740 TP: 88 FP: 4571 precision: 0.0189 recall: 1.0000 f1_score: 0.0371 f2_score: 0.0878
0 instances of class Chasis with average precision: 0.0000 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_sc

In [15]:
epoch = 100
last_model = os.path.join(SNAPSHOTS_DIR, 'resnet50_csv_{}.h5'.format(epoch))

!python ../../keras-retinanet/keras_retinanet/bin/train.py \
--config {CONFIG_FILE} \
--random-transform \
--weights {last_model} \
--initial-epoch {epoch} \
--weighted-average \
--batch-size {BATCH_SIZE} \
--steps {no_steps} \
--epochs 200 \
--no-resize \
--image-min-side 1000 \
--image-max-side 1000 \
--compute-val-loss \
--multiprocessing \
--workers 2 \
--tensorboard-dir {TENSORBOARD_LOGS_DIR} \
--snapshot-path {SNAPSHOTS_DIR} \
--validation-freq 5 \
--lr 1e-5 \
--reduce-lr-patience 5 \
--reduce-lr-factor 0.1 \
--gamma 3.0 \
--class-names '/home/aikauel/enap/aerialnet_project/datasets/data/v5/CLASSES_REPORT_NAMES' \
csv {TRAIN_ANNOTATIONS} {CLASSES_FILE} \
--val-annotations {VAL_ANNOTATIONS}

Using TensorFlow backend.
classNames: ['Animal', 'Basural-Escombro-MConstrucción', 'Bus', 'Camión', 'Chasis', 'Cilindro', 'Estructura', 'GHorquilla', 'Juegos', 'Maquinaria', 'PalletCaja', 'Persona', 'Pickup', 'Piscina', 'Poste', 'SAdvertencia', 'Tractor', 'Troncos', 'Tuberia', 'Vehículo']
Creating model, this may take a second...
2020-11-05 08:48:56.062847: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-11-05 08:48:56.094560: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-11-05 08:48:56.094919: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.755GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2020-11-05 08:48:56.0

tracking <tf.Variable 'Variable:0' shape=(27, 4) dtype=float32, numpy=
array([[ -57.984   ,  -14.496   ,   57.984   ,   14.496   ],
       [ -92.032   ,  -23.008   ,   92.032   ,   23.008   ],
       [-146.432   ,  -36.608   ,  146.432   ,   36.608   ],
       [ -45.556534,  -18.450396,   45.556534,   18.450396],
       [ -72.30717 ,  -29.284403,   72.30717 ,   29.284403],
       [-115.04785 ,  -46.59438 ,  115.04785 ,   46.59438 ],
       [ -38.707645,  -21.714989,   38.707645,   21.714989],
       [ -61.436634,  -34.465954,   61.436634,   34.465954],
       [ -97.75176 ,  -54.838737,   97.75176 ,   54.838737],
       [ -33.432533,  -25.141264,   33.432533,   25.141264],
       [ -53.063995,  -39.90412 ,   53.063995,   39.90412 ],
       [ -84.43006 ,  -63.491405,   84.43006 ,   63.491405],
       [ -28.992   ,  -28.992   ,   28.992   ,   28.992   ],
       [ -46.016   ,  -46.016   ,   46.016   ,   46.016   ],
       [ -73.216   ,  -73.216   ,   73.216   ,   73.216   ],
       [ -25.1

2020-11-05 08:48:57.868010: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1408] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI_ERROR_INSUFFICIENT_PRIVILEGES
2020-11-05 08:48:57.869802: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1447] function cupti_interface_->ActivityRegisterCallbacks( AllocCuptiActivityBuffer, FreeCuptiActivityBuffer)failed with error CUPTI_ERROR_INSUFFICIENT_PRIVILEGES
2020-11-05 08:48:57.869853: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1430] function cupti_interface_->EnableCallback( 0 , subscriber_, CUPTI_CB_DOMAIN_DRIVER_API, cbid)failed with error CUPTI_ERROR_INVALID_PARAMETER
Epoch 101/200
2020-11-05 08:49:13.663449: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2020-11-05 08:49:14.563908: W tensorflow/stream_executor/gpu/asm_compiler.cc:81] Running ptxas --version returned 256
2020-11-05 08

Epoch 102/200
646/646 [==============================] - 435s 673ms/step - loss: 0.3958 - regression_loss: 0.3078 - classification_loss: 0.0880
Running network: 100% (319 of 319) |#####| Elapsed Time: 0:00:31 Time:  0:00:31
Parsing annotations: 100% (319 of 319) |#| Elapsed Time: 0:00:00 Time:  0:00:00
99 instances of class Animal with average precision: 0.6793 TP: 83 FP: 994 precision: 0.0771 recall: 0.8384 f1_score: 0.1412 f2_score: 0.2817
65 instances of class Basural-Escombro-MConstrucción with average precision: 0.1613 TP: 43 FP: 1530 precision: 0.0273 recall: 0.6615 f1_score: 0.0525 f2_score: 0.1173
0 instances of class Bus with average precision: 0.0000 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_score: 0.0000 f2_score: 0.0000
88 instances of class Camión with average precision: 0.9270 TP: 88 FP: 4003 precision: 0.0215 recall: 1.0000 f1_score: 0.0421 f2_score: 0.0990
0 instances of class Chasis with average precision: 0.0000 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_sc

Epoch 105/200
646/646 [==============================] - 478s 739ms/step - loss: 0.3673 - regression_loss: 0.2907 - classification_loss: 0.0766 - val_loss: 0.4908 - val_regression_loss: 0.4449 - val_classification_loss: 0.1050
Running network: 100% (319 of 319) |#####| Elapsed Time: 0:00:30 Time:  0:00:30
Parsing annotations: 100% (319 of 319) |#| Elapsed Time: 0:00:00 Time:  0:00:00
99 instances of class Animal with average precision: 0.7427 TP: 85 FP: 768 precision: 0.0996 recall: 0.8586 f1_score: 0.1786 f2_score: 0.3403
65 instances of class Basural-Escombro-MConstrucción with average precision: 0.2053 TP: 43 FP: 1436 precision: 0.0291 recall: 0.6615 f1_score: 0.0557 f2_score: 0.1236
0 instances of class Bus with average precision: 0.0000 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_score: 0.0000 f2_score: 0.0000
88 instances of class Camión with average precision: 0.9346 TP: 88 FP: 2730 precision: 0.0312 recall: 1.0000 f1_score: 0.0606 f2_score: 0.1388
0 instances of class Chasi

...

Epoch 165/200
646/646 [==============================] - 489s 757ms/step - loss: 0.2260 - regression_loss: 0.1934 - classification_loss: 0.0325 - val_loss: 0.2573 - val_regression_loss: 0.3305 - val_classification_loss: 0.0899
Running network: 100% (319 of 319) |#####| Elapsed Time: 0:00:30 Time:  0:00:30
Parsing annotations: 100% (319 of 319) |#| Elapsed Time: 0:00:00 Time:  0:00:00
99 instances of class Animal with average precision: 0.7354 TP: 89 FP: 519 precision: 0.1464 recall: 0.8990 f1_score: 0.2518 f2_score: 0.4432
65 instances of class Basural-Escombro-MConstrucción with average precision: 0.2635 TP: 47 FP: 941 precision: 0.0476 recall: 0.7231 f1_score: 0.0893 f2_score: 0.1883
0 instances of class Bus with average precision: 0.0000 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_score: 0.0000 f2_score: 0.0000
88 instances of class Camión with average precision: 0.9732 TP: 88 FP: 297 precision: 0.2286 recall: 1.0000 f1_score: 0.3721 f2_score: 0.5970
0 instances of class Chasis 

Epoch 168/200
646/646 [==============================] - 454s 702ms/step - loss: 0.2163 - regression_loss: 0.1852 - classification_loss: 0.0310 - val_loss: 0.2573 - val_regression_loss: 0.3305 - val_classification_loss: 0.0899
Running network: 100% (319 of 319) |#####| Elapsed Time: 0:00:30 Time:  0:00:30
Parsing annotations: 100% (319 of 319) |#| Elapsed Time: 0:00:00 Time:  0:00:00
99 instances of class Animal with average precision: 0.7332 TP: 88 FP: 492 precision: 0.1517 recall: 0.8889 f1_score: 0.2592 f2_score: 0.4508
65 instances of class Basural-Escombro-MConstrucción with average precision: 0.2605 TP: 47 FP: 955 precision: 0.0469 recall: 0.7231 f1_score: 0.0881 f2_score: 0.1862
0 instances of class Bus with average precision: 0.0000 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_score: 0.0000 f2_score: 0.0000
88 instances of class Camión with average precision: 0.9720 TP: 88 FP: 302 precision: 0.2256 recall: 1.0000 f1_score: 0.3682 f2_score: 0.5930
0 instances of class Chasis 

Epoch 171/200
646/646 [==============================] - 451s 699ms/step - loss: 0.2196 - regression_loss: 0.1877 - classification_loss: 0.0319 - val_loss: 0.2478 - val_regression_loss: 0.3328 - val_classification_loss: 0.0908
Running network: 100% (319 of 319) |#####| Elapsed Time: 0:00:30 Time:  0:00:30
Parsing annotations: 100% (319 of 319) |#| Elapsed Time: 0:00:00 Time:  0:00:00
99 instances of class Animal with average precision: 0.7423 TP: 90 FP: 526 precision: 0.1461 recall: 0.9091 f1_score: 0.2517 f2_score: 0.4447
65 instances of class Basural-Escombro-MConstrucción with average precision: 0.2574 TP: 47 FP: 943 precision: 0.0475 recall: 0.7231 f1_score: 0.0891 f2_score: 0.1880
0 instances of class Bus with average precision: 0.0000 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_score: 0.0000 f2_score: 0.0000
88 instances of class Camión with average precision: 0.9729 TP: 88 FP: 278 precision: 0.2404 recall: 1.0000 f1_score: 0.3877 f2_score: 0.6128
0 instances of class Chasis 

Epoch 174/200
646/646 [==============================] - 454s 702ms/step - loss: 0.2144 - regression_loss: 0.1848 - classification_loss: 0.0296 - val_loss: 0.2478 - val_regression_loss: 0.3328 - val_classification_loss: 0.0908
Running network: 100% (319 of 319) |#####| Elapsed Time: 0:00:30 Time:  0:00:30
Parsing annotations: 100% (319 of 319) |#| Elapsed Time: 0:00:00 Time:  0:00:00
99 instances of class Animal with average precision: 0.7104 TP: 86 FP: 488 precision: 0.1498 recall: 0.8687 f1_score: 0.2556 f2_score: 0.4433
65 instances of class Basural-Escombro-MConstrucción with average precision: 0.2578 TP: 45 FP: 989 precision: 0.0435 recall: 0.6923 f1_score: 0.0819 f2_score: 0.1739
0 instances of class Bus with average precision: 0.0000 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_score: 0.0000 f2_score: 0.0000
88 instances of class Camión with average precision: 0.9758 TP: 88 FP: 286 precision: 0.2353 recall: 1.0000 f1_score: 0.3810 f2_score: 0.6061
0 instances of class Chasis 

Epoch 177/200
646/646 [==============================] - 454s 702ms/step - loss: 0.2196 - regression_loss: 0.1889 - classification_loss: 0.0307 - val_loss: 0.2250 - val_regression_loss: 0.3339 - val_classification_loss: 0.0921
Running network: 100% (319 of 319) |#####| Elapsed Time: 0:00:30 Time:  0:00:30
Parsing annotations: 100% (319 of 319) |#| Elapsed Time: 0:00:00 Time:  0:00:00
99 instances of class Animal with average precision: 0.7293 TP: 90 FP: 514 precision: 0.1490 recall: 0.9091 f1_score: 0.2560 f2_score: 0.4500
65 instances of class Basural-Escombro-MConstrucción with average precision: 0.2802 TP: 46 FP: 974 precision: 0.0451 recall: 0.7077 f1_score: 0.0848 f2_score: 0.1797
0 instances of class Bus with average precision: 0.0000 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_score: 0.0000 f2_score: 0.0000
88 instances of class Camión with average precision: 0.9798 TP: 88 FP: 282 precision: 0.2378 recall: 1.0000 f1_score: 0.3843 f2_score: 0.6094
0 instances of class Chasis 

## Test model
#### Se procesan las imágenes del dataset de testing

In [24]:
# Convert training model to inference model
# path to trained weights
epoch = 153
last_model = os.path.join(SNAPSHOTS_DIR, 'resnet50_csv_{}.h5'.format(epoch))
inference_model = last_model.replace('snapshots', 'inference')

# convert trained model to inference model to be used for generating predictions
inference_model = last_model.replace('snapshots/', 'inference/')
!python ../../keras-retinanet/keras_retinanet/bin/convert_model.py {last_model} {inference_model} \
--config {CONFIG_FILE}

Using TensorFlow backend.
2020-11-05 21:54:32.291415: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-11-05 21:54:32.323743: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-11-05 21:54:32.324087: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.755GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2020-11-05 21:54:32.324221: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-11-05 21:54:32.325173: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2020-11-05 21:54:32.326131: I tensorf

tracking <tf.Variable 'Variable:0' shape=(27, 4) dtype=float32, numpy=
array([[-115.968   ,  -28.992   ,  115.968   ,   28.992   ],
       [-184.064   ,  -46.016   ,  184.064   ,   46.016   ],
       [-292.864   ,  -73.216   ,  292.864   ,   73.216   ],
       [ -91.11307 ,  -36.90079 ,   91.11307 ,   36.90079 ],
       [-144.61433 ,  -58.568806,  144.61433 ,   58.568806],
       [-230.0957  ,  -93.18876 ,  230.0957  ,   93.18876 ],
       [ -77.41529 ,  -43.429977,   77.41529 ,   43.429977],
       [-122.87327 ,  -68.93191 ,  122.87327 ,   68.93191 ],
       [-195.50352 , -109.677475,  195.50352 ,  109.677475],
       [ -66.86507 ,  -50.282528,   66.86507 ,   50.282528],
       [-106.12799 ,  -79.80824 ,  106.12799 ,   79.80824 ],
       [-168.86012 , -126.98281 ,  168.86012 ,  126.98281 ],
       [ -57.984   ,  -57.984   ,   57.984   ,   57.984   ],
       [ -92.032   ,  -92.032   ,   92.032   ,   92.032   ],
       [-146.432   , -146.432   ,  146.432   ,  146.432   ],
       [ -50.2

#### Set paths

In [25]:
test_images_path = '/home/aikauel/enap/aerialnet_project/datasets/official_37_test_images_ori/'
#test_images_path = '/home/aikauel/enap/data/vitto/'
test_output_path = os.path.join(TRAINING_BASE_DIR, 
                                'official_37_output_epoch{}_threshold53'.format(epoch))

In [26]:
test_output_path

'../trainings/v5/official_37_output_epoch153_threshold53'

In [27]:
!python ../../aerialnet_project/shared_utils/image_inference_write.py \
--input_dir {test_images_path} \
--model {inference_model} \
--output_dir {test_output_path} \
--labels {CLASSES_FILE} \
--threshold 0.53 \
--no_csv True

Using TensorFlow backend.
/home/aikauel/enap/aerialnet_project/datasets/official_37_test_images_ori/
LABELS: {0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9', 10: '10', 11: '11', 12: '12', 13: '13', 14: '14', 15: '15', 16: '16', 17: '17', 18: '18', 19: '19'}
2020-11-05 21:54:41.446361: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-11-05 21:54:41.478252: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-11-05 21:54:41.478559: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.755GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2020-11-05 21:54:41.478689: I tensorflow/stream_executor/platfo

tracking <tf.Variable 'Variable:0' shape=(27, 4) dtype=float32, numpy=
array([[-115.968   ,  -28.992   ,  115.968   ,   28.992   ],
       [-184.064   ,  -46.016   ,  184.064   ,   46.016   ],
       [-292.864   ,  -73.216   ,  292.864   ,   73.216   ],
       [ -91.11307 ,  -36.90079 ,   91.11307 ,   36.90079 ],
       [-144.61433 ,  -58.568806,  144.61433 ,   58.568806],
       [-230.0957  ,  -93.18876 ,  230.0957  ,   93.18876 ],
       [ -77.41529 ,  -43.429977,   77.41529 ,   43.429977],
       [-122.87327 ,  -68.93191 ,  122.87327 ,   68.93191 ],
       [-195.50352 , -109.677475,  195.50352 ,  109.677475],
       [ -66.86507 ,  -50.282528,   66.86507 ,   50.282528],
       [-106.12799 ,  -79.80824 ,  106.12799 ,   79.80824 ],
       [-168.86012 , -126.98281 ,  168.86012 ,  126.98281 ],
       [ -57.984   ,  -57.984   ,   57.984   ,   57.984   ],
       [ -92.032   ,  -92.032   ,   92.032   ,   92.032   ],
       [-146.432   , -146.432   ,  146.432   ,  146.432   ],
       [ -50.2

[INFO] predicting image 19 of 37: 2020-08-11_09:43:10-3716425-DJI_0341.jpg
processing time:  0.46580076217651367
../trainings/v5/official_37_output_epoch153_threshold53/2020-08-11_09:43:10-3716425-DJI_0341.jpg
[INFO] predicting image 20 of 37: 2020-08-11_08:51:32-3649235-DJI_0340.jpg
processing time:  0.46973133087158203
../trainings/v5/official_37_output_epoch153_threshold53/2020-08-11_08:51:32-3649235-DJI_0340.jpg
[INFO] predicting image 21 of 37: 2020-08-11_08:16:00-3566645-DJI_0298.jpg
processing time:  0.4839749336242676
../trainings/v5/official_37_output_epoch153_threshold53/2020-08-11_08:16:00-3566645-DJI_0298.jpg
[INFO] predicting image 22 of 37: 2020-08-11_08:13:14-3656619-DJI_0280.jpg
processing time:  0.4788689613342285
../trainings/v5/official_37_output_epoch153_threshold53/2020-08-11_08:13:14-3656619-DJI_0280.jpg
[INFO] predicting image 23 of 37: 2020-08-11_08:52:04-3632516-DJI_0341.jpg
processing time:  0.5002806186676025
../trainings/v5/official_37_output_epoch153_thresh

# ------------------  BEST EPOCH: 157  -------------------

# Iteración 3:
    # Se incluyen ejemplos del dataset de SkyNet más los del dataset de falsos positivos de Septiembre (solo las clases más relevantes)

In [1]:
## Set paths
import os
DATASET_BASE_DIR = '../datasets/data'
DATASET_VERSION = 'v5'
DATASET_VERSION_DIR = os.path.join(DATASET_BASE_DIR, DATASET_VERSION)
CLASSES_FILE = os.path.join(DATASET_VERSION_DIR, 'classes.csv')
TRAIN_ANNOTATIONS = os.path.join(DATASET_VERSION_DIR, 'train_iter3.csv')
VAL_ANNOTATIONS = os.path.join(DATASET_VERSION_DIR, 'val_iter3.csv')

TRAINING_BASE_DIR = os.path.join('../trainings', DATASET_VERSION)

## Check directories and files

In [2]:
!wc -l {TRAIN_ANNOTATIONS}

6369 ../datasets/data/v5/train_iter3.csv


In [3]:
!wc -l {VAL_ANNOTATIONS}

832 ../datasets/data/v5/val_iter3.csv


In [4]:
!wc -l {CLASSES_FILE}

19 ../datasets/data/v5/classes.csv


# Set number of steps per epoch

# Train model

In [6]:
CONFIG_FILE = os.path.join(TRAINING_BASE_DIR, 'config.ini')
TENSORBOARD_LOGS_DIR = os.path.join(TRAINING_BASE_DIR, 'logs')
SNAPSHOTS_DIR = os.path.join(TRAINING_BASE_DIR, 'snapshots')

#### Full model training

In [33]:
BATCH_SIZE = 4

# compute number of annotations to compute number of steps per epoch
df = pd.read_csv(TRAIN_ANNOTATIONS, header=None, names=['img_path', 'x1', 'y1', 'x2', 'y2', 'class'])

countAnn = len(df)-1
countImg = df['img_path'].nunique()
no_steps = math.ceil(countImg/BATCH_SIZE)

print("Count of images: {}".format(countImg))
print("Count of annotations: {}".format(countAnn))
print("Number of steps per epoch: {}".format(no_steps))

Count of images: 3917
Count of annotations: 6294
Number of steps per epoch: 980


In [34]:
epoch = 157
last_model = os.path.join(SNAPSHOTS_DIR, 'resnet50_csv_{}.h5'.format(epoch))

!python ../../keras-retinanet/keras_retinanet/bin/train.py \
--config {CONFIG_FILE} \
--random-transform \
--weights {last_model} \
--initial-epoch {epoch} \
--weighted-average \
--batch-size {BATCH_SIZE} \
--steps {no_steps} \
--epochs 200 \
--no-resize \
--image-min-side 1000 \
--image-max-side 1000 \
--compute-val-loss \
--multiprocessing \
--workers 2 \
--tensorboard-dir {TENSORBOARD_LOGS_DIR} \
--snapshot-path {SNAPSHOTS_DIR} \
--validation-freq 5 \
--lr 1e-5 \
--reduce-lr-patience 5 \
--reduce-lr-factor 0.1 \
--gamma 3.0 \
--class-names '/home/aikauel/enap/aerialnet_project/datasets/data/v5/CLASSES_REPORT_NAMES' \
csv {TRAIN_ANNOTATIONS} {CLASSES_FILE} \
--val-annotations {VAL_ANNOTATIONS}

Using TensorFlow backend.
classNames: ['Animal', 'Basural-Escombro-MConstrucción', 'Bus', 'Camión', 'Chasis', 'Cilindro', 'Estructura', 'GHorquilla', 'Juegos', 'Maquinaria', 'PalletCaja', 'Persona', 'Pickup', 'Piscina', 'Poste', 'SAdvertencia', 'Tractor', 'Troncos', 'Tuberia', 'Vehículo']
Creating model, this may take a second...
2020-11-06 17:40:01.407696: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-11-06 17:40:01.436483: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-11-06 17:40:01.436783: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.755GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2020-11-06 17:40:01.4

tracking <tf.Variable 'Variable:0' shape=(27, 4) dtype=float32, numpy=
array([[ -57.984   ,  -14.496   ,   57.984   ,   14.496   ],
       [ -92.032   ,  -23.008   ,   92.032   ,   23.008   ],
       [-146.432   ,  -36.608   ,  146.432   ,   36.608   ],
       [ -45.556534,  -18.450396,   45.556534,   18.450396],
       [ -72.30717 ,  -29.284403,   72.30717 ,   29.284403],
       [-115.04785 ,  -46.59438 ,  115.04785 ,   46.59438 ],
       [ -38.707645,  -21.714989,   38.707645,   21.714989],
       [ -61.436634,  -34.465954,   61.436634,   34.465954],
       [ -97.75176 ,  -54.838737,   97.75176 ,   54.838737],
       [ -33.432533,  -25.141264,   33.432533,   25.141264],
       [ -53.063995,  -39.90412 ,   53.063995,   39.90412 ],
       [ -84.43006 ,  -63.491405,   84.43006 ,   63.491405],
       [ -28.992   ,  -28.992   ,   28.992   ,   28.992   ],
       [ -46.016   ,  -46.016   ,   46.016   ,   46.016   ],
       [ -73.216   ,  -73.216   ,   73.216   ,   73.216   ],
       [ -25.1

2020-11-06 17:40:03.220601: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1408] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI_ERROR_INSUFFICIENT_PRIVILEGES
2020-11-06 17:40:03.222485: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1447] function cupti_interface_->ActivityRegisterCallbacks( AllocCuptiActivityBuffer, FreeCuptiActivityBuffer)failed with error CUPTI_ERROR_INSUFFICIENT_PRIVILEGES
2020-11-06 17:40:03.222543: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1430] function cupti_interface_->EnableCallback( 0 , subscriber_, CUPTI_CB_DOMAIN_DRIVER_API, cbid)failed with error CUPTI_ERROR_INVALID_PARAMETER
Epoch 158/200
2020-11-06 17:40:19.046341: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2020-11-06 17:40:19.915621: W tensorflow/stream_executor/gpu/asm_compiler.cc:81] Running ptxas --version returned 256
2020-11-06 17

Epoch 159/200
980/980 [==============================] - 670s 684ms/step - loss: 0.2320 - regression_loss: 0.1664 - classification_loss: 0.0656
Running network: 100% (524 of 524) |#####| Elapsed Time: 0:00:49 Time:  0:00:49
Parsing annotations: 100% (524 of 524) |#| Elapsed Time: 0:00:00 Time:  0:00:00
99 instances of class Animal with average precision: 0.6340 TP: 84 FP: 1596 precision: 0.0500 recall: 0.8485 f1_score: 0.0944 f2_score: 0.2023
65 instances of class Basural-Escombro-MConstrucción with average precision: 0.1566 TP: 44 FP: 1291 precision: 0.0330 recall: 0.6769 f1_score: 0.0629 f2_score: 0.1379
0 instances of class Bus with average precision: 0.0000 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_score: 0.0000 f2_score: 0.0000
139 instances of class Camión with average precision: 0.9648 TP: 139 FP: 500 precision: 0.2175 recall: 1.0000 f1_score: 0.3573 f2_score: 0.5816
69 instances of class Chasis with average precision: 0.8294 TP: 69 FP: 5709 precision: 0.0119 recall: 1.000

Epoch 162/200
980/980 [==============================] - 671s 684ms/step - loss: 0.2212 - regression_loss: 0.1665 - classification_loss: 0.0547 - val_loss: 0.2532 - val_regression_loss: 0.2489 - val_classification_loss: 0.1034
Running network: 100% (524 of 524) |#####| Elapsed Time: 0:00:49 Time:  0:00:49
Parsing annotations: 100% (524 of 524) |#| Elapsed Time: 0:00:00 Time:  0:00:00
99 instances of class Animal with average precision: 0.7286 TP: 87 FP: 1209 precision: 0.0671 recall: 0.8788 f1_score: 0.1247 f2_score: 0.2571
65 instances of class Basural-Escombro-MConstrucción with average precision: 0.1692 TP: 42 FP: 1097 precision: 0.0369 recall: 0.6462 f1_score: 0.0698 f2_score: 0.1501
0 instances of class Bus with average precision: 0.0000 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_score: 0.0000 f2_score: 0.0000
139 instances of class Camión with average precision: 0.9630 TP: 139 FP: 687 precision: 0.1683 recall: 1.0000 f1_score: 0.2881 f2_score: 0.5029
69 instances of class Ch

...

Epoch 186/200
980/980 [==============================] - 682s 696ms/step - loss: 0.1893 - regression_loss: 0.1551 - classification_loss: 0.0342 - val_loss: 0.0839 - val_regression_loss: 0.2184 - val_classification_loss: 0.0836
Running network: 100% (524 of 524) |#####| Elapsed Time: 0:00:49 Time:  0:00:49
Parsing annotations: 100% (524 of 524) |#| Elapsed Time: 0:00:00 Time:  0:00:00
99 instances of class Animal with average precision: 0.7385 TP: 88 FP: 769 precision: 0.1027 recall: 0.8889 f1_score: 0.1841 f2_score: 0.3512
65 instances of class Basural-Escombro-MConstrucción with average precision: 0.2170 TP: 48 FP: 927 precision: 0.0492 recall: 0.7385 f1_score: 0.0923 f2_score: 0.1943
0 instances of class Bus with average precision: 0.0000 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_score: 0.0000 f2_score: 0.0000
139 instances of class Camión with average precision: 0.9705 TP: 138 FP: 287 precision: 0.3247 recall: 0.9928 f1_score: 0.4894 f2_score: 0.7034
69 instances of class Chas

Epoch 189/200
980/980 [==============================] - 685s 699ms/step - loss: 0.1826 - regression_loss: 0.1512 - classification_loss: 0.0314 - val_loss: 0.0839 - val_regression_loss: 0.2184 - val_classification_loss: 0.0836
Running network: 100% (524 of 524) |#####| Elapsed Time: 0:00:49 Time:  0:00:49
Parsing annotations: 100% (524 of 524) |#| Elapsed Time: 0:00:00 Time:  0:00:00
99 instances of class Animal with average precision: 0.6930 TP: 82 FP: 719 precision: 0.1024 recall: 0.8283 f1_score: 0.1822 f2_score: 0.3425
65 instances of class Basural-Escombro-MConstrucción with average precision: 0.2178 TP: 40 FP: 879 precision: 0.0435 recall: 0.6154 f1_score: 0.0813 f2_score: 0.1696
0 instances of class Bus with average precision: 0.0000 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_score: 0.0000 f2_score: 0.0000
139 instances of class Camión with average precision: 0.9816 TP: 139 FP: 347 precision: 0.2860 recall: 1.0000 f1_score: 0.4448 f2_score: 0.6670
69 instances of class Chas

Epoch 192/200
980/980 [==============================] - 686s 700ms/step - loss: 0.1827 - regression_loss: 0.1513 - classification_loss: 0.0314 - val_loss: 0.0824 - val_regression_loss: 0.2383 - val_classification_loss: 0.0921
Running network: 100% (524 of 524) |#####| Elapsed Time: 0:00:49 Time:  0:00:49
Parsing annotations: 100% (524 of 524) |#| Elapsed Time: 0:00:00 Time:  0:00:00
99 instances of class Animal with average precision: 0.7591 TP: 90 FP: 778 precision: 0.1037 recall: 0.9091 f1_score: 0.1861 f2_score: 0.3560
65 instances of class Basural-Escombro-MConstrucción with average precision: 0.2305 TP: 46 FP: 1006 precision: 0.0437 recall: 0.7077 f1_score: 0.0824 f2_score: 0.1753
0 instances of class Bus with average precision: 0.0000 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_score: 0.0000 f2_score: 0.0000
139 instances of class Camión with average precision: 0.9742 TP: 138 FP: 298 precision: 0.3165 recall: 0.9928 f1_score: 0.4800 f2_score: 0.6956
69 instances of class Cha

Epoch 195/200
980/980 [==============================] - 742s 757ms/step - loss: 0.1800 - regression_loss: 0.1498 - classification_loss: 0.0302 - val_loss: 0.0868 - val_regression_loss: 0.2406 - val_classification_loss: 0.0934
Running network: 100% (524 of 524) |#####| Elapsed Time: 0:00:49 Time:  0:00:49
Parsing annotations: 100% (524 of 524) |#| Elapsed Time: 0:00:00 Time:  0:00:00
99 instances of class Animal with average precision: 0.7293 TP: 91 FP: 775 precision: 0.1051 recall: 0.9192 f1_score: 0.1886 f2_score: 0.3605
65 instances of class Basural-Escombro-MConstrucción with average precision: 0.2034 TP: 46 FP: 1068 precision: 0.0413 recall: 0.7077 f1_score: 0.0780 f2_score: 0.1674
0 instances of class Bus with average precision: 0.0000 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_score: 0.0000 f2_score: 0.0000
139 instances of class Camión with average precision: 0.9794 TP: 139 FP: 267 precision: 0.3424 recall: 1.0000 f1_score: 0.5101 f2_score: 0.7225
69 instances of class Cha

Epoch 198/200
980/980 [==============================] - 685s 699ms/step - loss: 0.1721 - regression_loss: 0.1446 - classification_loss: 0.0275 - val_loss: 0.0868 - val_regression_loss: 0.2406 - val_classification_loss: 0.0934
Running network: 100% (524 of 524) |#####| Elapsed Time: 0:00:49 Time:  0:00:49
Parsing annotations: 100% (524 of 524) |#| Elapsed Time: 0:00:00 Time:  0:00:00
99 instances of class Animal with average precision: 0.7691 TP: 89 FP: 790 precision: 0.1013 recall: 0.8990 f1_score: 0.1820 f2_score: 0.3490
65 instances of class Basural-Escombro-MConstrucción with average precision: 0.2176 TP: 37 FP: 852 precision: 0.0416 recall: 0.5692 f1_score: 0.0776 f2_score: 0.1610
0 instances of class Bus with average precision: 0.0000 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_score: 0.0000 f2_score: 0.0000
139 instances of class Camión with average precision: 0.9822 TP: 138 FP: 302 precision: 0.3136 recall: 0.9928 f1_score: 0.4767 f2_score: 0.6928
69 instances of class Chas

## Test model
#### Se procesan las imágenes del dataset de testing

In [40]:
# Convert training model to inference model
# path to trained weights
epoch = 191
last_model = os.path.join(SNAPSHOTS_DIR, 'resnet50_csv_{}.h5'.format(epoch))
inference_model = last_model.replace('snapshots', 'inference')

# convert trained model to inference model to be used for generating predictions
inference_model = last_model.replace('snapshots/', 'inference/')
!python ../../keras-retinanet/keras_retinanet/bin/convert_model.py {last_model} {inference_model} \
--config {CONFIG_FILE}

Using TensorFlow backend.
2020-11-07 19:50:07.254103: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-11-07 19:50:07.285722: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-11-07 19:50:07.286059: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.755GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2020-11-07 19:50:07.286208: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-11-07 19:50:07.287253: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2020-11-07 19:50:07.288241: I tensorf

tracking <tf.Variable 'Variable:0' shape=(27, 4) dtype=float32, numpy=
array([[-115.968   ,  -28.992   ,  115.968   ,   28.992   ],
       [-184.064   ,  -46.016   ,  184.064   ,   46.016   ],
       [-292.864   ,  -73.216   ,  292.864   ,   73.216   ],
       [ -91.11307 ,  -36.90079 ,   91.11307 ,   36.90079 ],
       [-144.61433 ,  -58.568806,  144.61433 ,   58.568806],
       [-230.0957  ,  -93.18876 ,  230.0957  ,   93.18876 ],
       [ -77.41529 ,  -43.429977,   77.41529 ,   43.429977],
       [-122.87327 ,  -68.93191 ,  122.87327 ,   68.93191 ],
       [-195.50352 , -109.677475,  195.50352 ,  109.677475],
       [ -66.86507 ,  -50.282528,   66.86507 ,   50.282528],
       [-106.12799 ,  -79.80824 ,  106.12799 ,   79.80824 ],
       [-168.86012 , -126.98281 ,  168.86012 ,  126.98281 ],
       [ -57.984   ,  -57.984   ,   57.984   ,   57.984   ],
       [ -92.032   ,  -92.032   ,   92.032   ,   92.032   ],
       [-146.432   , -146.432   ,  146.432   ,  146.432   ],
       [ -50.2

#### Set paths

In [41]:
test_images_path = '/home/aikauel/enap/aerialnet_project/datasets/official_37_test_images_ori/'
#test_images_path = '/home/aikauel/enap/data/vitto/'
test_output_path = os.path.join(TRAINING_BASE_DIR, 
                                'official_37_output_epoch{}_threshold53'.format(epoch))

In [42]:
test_output_path

'../trainings/v5/official_37_output_epoch191_threshold53'

In [43]:
!python ../../aerialnet_project/shared_utils/image_inference_write.py \
--input_dir {test_images_path} \
--model {inference_model} \
--output_dir {test_output_path} \
--labels {CLASSES_FILE} \
--threshold 0.53 \
--no_csv True

Using TensorFlow backend.
/home/aikauel/enap/aerialnet_project/datasets/official_37_test_images_ori/
LABELS: {0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9', 10: '10', 11: '11', 12: '12', 13: '13', 14: '14', 15: '15', 16: '16', 17: '17', 18: '18', 19: '19'}
2020-11-07 19:50:16.144598: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-11-07 19:50:16.173374: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-11-07 19:50:16.173701: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.755GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2020-11-07 19:50:16.173854: I tensorflow/stream_executor/platfo

/home/aikauel/anaconda3/envs/enap/lib/python3.6/site-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
no csv: /home/aikauel/enap/aerialnet_project/datasets/official_37_test_images_ori/
[INFO] predicting image 1 of 37: 2020-08-11_08:13:26-3872762-DJI_0282.jpg
2020-11-07 19:50:19.495893: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2020-11-07 19:50:20.134065: W tensorflow/stream_executor/gpu/asm_compiler.cc:81] Running ptxas --version returned 256
2020-11-07 19:50:20.179917: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2020-11-07 19:50:21.313823: I tensorflow/

[FINAL] Predictions completed!


# ------------------  BEST EPOCH: 191  -------------------

In [33]:
## Set paths
import os
DATASET_BASE_DIR = '../datasets/data'
DATASET_VERSION = 'v5'
DATASET_VERSION_DIR = os.path.join(DATASET_BASE_DIR, DATASET_VERSION)
CLASSES_FILE = os.path.join(DATASET_VERSION_DIR, 'classes.csv')
TRAIN_ANNOTATIONS = os.path.join(DATASET_VERSION_DIR, 'train_iter3.csv')
VAL_ANNOTATIONS = os.path.join(DATASET_VERSION_DIR, 'val_iter3.csv')

TRAINING_BASE_DIR = os.path.join('../trainings', DATASET_VERSION)

## Check directories and files

In [34]:
!wc -l {TRAIN_ANNOTATIONS}

6412 ../datasets/data/v5/train_iter3.csv


In [35]:
!wc -l {VAL_ANNOTATIONS}

789 ../datasets/data/v5/val_iter3.csv


In [36]:
!wc -l {CLASSES_FILE}

19 ../datasets/data/v5/classes.csv


# Set number of steps per epoch

# Train model

In [37]:
CONFIG_FILE = os.path.join(TRAINING_BASE_DIR, 'config.ini')
TENSORBOARD_LOGS_DIR = os.path.join(TRAINING_BASE_DIR, 'logs')
SNAPSHOTS_DIR = os.path.join(TRAINING_BASE_DIR, 'snapshots')

#### Full model training

In [38]:
BATCH_SIZE = 4

# compute number of annotations to compute number of steps per epoch
df = pd.read_csv(TRAIN_ANNOTATIONS, header=None, names=['img_path', 'x1', 'y1', 'x2', 'y2', 'class'])

countAnn = len(df)-1
countImg = df['img_path'].nunique()
no_steps = math.ceil(countImg/BATCH_SIZE)

print("Count of images: {}".format(countImg))
print("Count of annotations: {}".format(countAnn))
print("Number of steps per epoch: {}".format(no_steps))

Count of images: 4035
Count of annotations: 6412
Number of steps per epoch: 1009


In [39]:
epoch = 191
last_model = os.path.join(SNAPSHOTS_DIR, 'resnet50_csv_{}.h5'.format(epoch))

!python ../../keras-retinanet/keras_retinanet/bin/train.py \
--config {CONFIG_FILE} \
--random-transform \
--weights {last_model} \
--initial-epoch {epoch} \
--weighted-average \
--batch-size {BATCH_SIZE} \
--steps {no_steps} \
--epochs 300 \
--no-resize \
--image-min-side 1000 \
--image-max-side 1000 \
--compute-val-loss \
--multiprocessing \
--workers 2 \
--tensorboard-dir {TENSORBOARD_LOGS_DIR} \
--snapshot-path {SNAPSHOTS_DIR} \
--validation-freq 5 \
--lr 1e-5 \
--reduce-lr-patience 5 \
--reduce-lr-factor 0.1 \
--gamma 3.0 \
--class-names '/home/aikauel/enap/aerialnet_project/datasets/data/v5/CLASSES_REPORT_NAMES' \
csv {TRAIN_ANNOTATIONS} {CLASSES_FILE} \
--val-annotations {VAL_ANNOTATIONS}

Using TensorFlow backend.
classNames: ['Animal', 'Basural-Escombro-MConstrucción', 'Bus', 'Camión', 'Chasis', 'Cilindro', 'Estructura', 'GHorquilla', 'Juegos', 'Maquinaria', 'PalletCaja', 'Persona', 'Pickup', 'Piscina', 'Poste', 'SAdvertencia', 'Tractor', 'Troncos', 'Tuberia', 'Vehículo']
Creating model, this may take a second...
2020-11-11 02:19:15.420915: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-11-11 02:19:15.468091: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-11-11 02:19:15.468614: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.755GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2020-11-11 02:19:15.4

tracking <tf.Variable 'Variable:0' shape=(27, 4) dtype=float32, numpy=
array([[ -57.984   ,  -14.496   ,   57.984   ,   14.496   ],
       [ -92.032   ,  -23.008   ,   92.032   ,   23.008   ],
       [-146.432   ,  -36.608   ,  146.432   ,   36.608   ],
       [ -45.556534,  -18.450396,   45.556534,   18.450396],
       [ -72.30717 ,  -29.284403,   72.30717 ,   29.284403],
       [-115.04785 ,  -46.59438 ,  115.04785 ,   46.59438 ],
       [ -38.707645,  -21.714989,   38.707645,   21.714989],
       [ -61.436634,  -34.465954,   61.436634,   34.465954],
       [ -97.75176 ,  -54.838737,   97.75176 ,   54.838737],
       [ -33.432533,  -25.141264,   33.432533,   25.141264],
       [ -53.063995,  -39.90412 ,   53.063995,   39.90412 ],
       [ -84.43006 ,  -63.491405,   84.43006 ,   63.491405],
       [ -28.992   ,  -28.992   ,   28.992   ,   28.992   ],
       [ -46.016   ,  -46.016   ,   46.016   ,   46.016   ],
       [ -73.216   ,  -73.216   ,   73.216   ,   73.216   ],
       [ -25.1

2020-11-11 02:19:17.642011: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1408] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI_ERROR_INSUFFICIENT_PRIVILEGES
2020-11-11 02:19:17.643860: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1447] function cupti_interface_->ActivityRegisterCallbacks( AllocCuptiActivityBuffer, FreeCuptiActivityBuffer)failed with error CUPTI_ERROR_INSUFFICIENT_PRIVILEGES
2020-11-11 02:19:17.643914: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1430] function cupti_interface_->EnableCallback( 0 , subscriber_, CUPTI_CB_DOMAIN_DRIVER_API, cbid)failed with error CUPTI_ERROR_INVALID_PARAMETER
Epoch 192/300
2020-11-11 02:19:33.967272: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2020-11-11 02:19:34.904122: W tensorflow/stream_executor/gpu/asm_compiler.cc:81] Running ptxas --version returned 256
2020-11-11 02

Epoch 193/300
1009/1009 [==============================] - 683s 677ms/step - loss: 0.1869 - regression_loss: 0.1462 - classification_loss: 0.0407
Running network: 100% (524 of 524) |#####| Elapsed Time: 0:00:49 Time:  0:00:49
Parsing annotations: 100% (524 of 524) |#| Elapsed Time: 0:00:00 Time:  0:00:00
99 instances of class Animal with average precision: 0.6229 TP: 80 FP: 805 precision: 0.0904 recall: 0.8081 f1_score: 0.1626 f2_score: 0.3123
65 instances of class Basural-Escombro-MConstrucción with average precision: 0.1878 TP: 43 FP: 1003 precision: 0.0411 recall: 0.6615 f1_score: 0.0774 f2_score: 0.1646
0 instances of class Bus with average precision: 0.0000 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_score: 0.0000 f2_score: 0.0000
139 instances of class Camión with average precision: 0.9812 TP: 139 FP: 292 precision: 0.3225 recall: 1.0000 f1_score: 0.4877 f2_score: 0.7042
69 instances of class Chasis with average precision: 0.9685 TP: 69 FP: 531 precision: 0.1150 recall: 1.000

Epoch 196/300
1009/1009 [==============================] - 682s 676ms/step - loss: 0.1808 - regression_loss: 0.1429 - classification_loss: 0.0378 - val_loss: 0.1072 - val_regression_loss: 0.2343 - val_classification_loss: 0.0954
Running network: 100% (524 of 524) |#####| Elapsed Time: 0:00:49 Time:  0:00:49
Parsing annotations: 100% (524 of 524) |#| Elapsed Time: 0:00:00 Time:  0:00:00
99 instances of class Animal with average precision: 0.7445 TP: 88 FP: 656 precision: 0.1183 recall: 0.8889 f1_score: 0.2088 f2_score: 0.3860
65 instances of class Basural-Escombro-MConstrucción with average precision: 0.2532 TP: 43 FP: 1019 precision: 0.0405 recall: 0.6615 f1_score: 0.0763 f2_score: 0.1626
0 instances of class Bus with average precision: 0.0000 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_score: 0.0000 f2_score: 0.0000
139 instances of class Camión with average precision: 0.9846 TP: 139 FP: 317 precision: 0.3048 recall: 1.0000 f1_score: 0.4672 f2_score: 0.6868
69 instances of class C

...

Epoch 235/300
1009/1009 [==============================] - 741s 735ms/step - loss: 0.1410 - regression_loss: 0.1207 - classification_loss: 0.0203 - val_loss: 0.0517 - val_regression_loss: 0.2080 - val_classification_loss: 0.1011
Running network: 100% (524 of 524) |#####| Elapsed Time: 0:00:49 Time:  0:00:49
Parsing annotations: 100% (524 of 524) |#| Elapsed Time: 0:00:00 Time:  0:00:00
99 instances of class Animal with average precision: 0.7444 TP: 85 FP: 647 precision: 0.1161 recall: 0.8586 f1_score: 0.2046 f2_score: 0.3768
65 instances of class Basural-Escombro-MConstrucción with average precision: 0.2504 TP: 47 FP: 1035 precision: 0.0434 recall: 0.7231 f1_score: 0.0820 f2_score: 0.1751
0 instances of class Bus with average precision: 0.0000 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_score: 0.0000 f2_score: 0.0000
139 instances of class Camión with average precision: 0.9786 TP: 138 FP: 177 precision: 0.4381 recall: 0.9928 f1_score: 0.6079 f2_score: 0.7922
69 instances of class C

Epoch 238/300
1009/1009 [==============================] - 683s 677ms/step - loss: 0.1370 - regression_loss: 0.1182 - classification_loss: 0.0188 - val_loss: 0.0517 - val_regression_loss: 0.2080 - val_classification_loss: 0.1011
Running network: 100% (524 of 524) |#####| Elapsed Time: 0:00:49 Time:  0:00:49
Parsing annotations: 100% (524 of 524) |#| Elapsed Time: 0:00:00 Time:  0:00:00
99 instances of class Animal with average precision: 0.7400 TP: 85 FP: 649 precision: 0.1158 recall: 0.8586 f1_score: 0.2041 f2_score: 0.3761
65 instances of class Basural-Escombro-MConstrucción with average precision: 0.2490 TP: 46 FP: 1018 precision: 0.0432 recall: 0.7077 f1_score: 0.0815 f2_score: 0.1737
0 instances of class Bus with average precision: 0.0000 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_score: 0.0000 f2_score: 0.0000
139 instances of class Camión with average precision: 0.9782 TP: 138 FP: 170 precision: 0.4481 recall: 0.9928 f1_score: 0.6174 f2_score: 0.7986
69 instances of class C

Epoch 241/300
1009/1009 [==============================] - 681s 675ms/step - loss: 0.1394 - regression_loss: 0.1196 - classification_loss: 0.0198 - val_loss: 0.0564 - val_regression_loss: 0.2102 - val_classification_loss: 0.1022
Running network: 100% (524 of 524) |#####| Elapsed Time: 0:00:49 Time:  0:00:49
Parsing annotations: 100% (524 of 524) |#| Elapsed Time: 0:00:00 Time:  0:00:00
99 instances of class Animal with average precision: 0.7425 TP: 86 FP: 641 precision: 0.1183 recall: 0.8687 f1_score: 0.2082 f2_score: 0.3829
65 instances of class Basural-Escombro-MConstrucción with average precision: 0.2497 TP: 46 FP: 1015 precision: 0.0434 recall: 0.7077 f1_score: 0.0817 f2_score: 0.1741
0 instances of class Bus with average precision: 0.0000 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_score: 0.0000 f2_score: 0.0000
139 instances of class Camión with average precision: 0.9780 TP: 138 FP: 173 precision: 0.4437 recall: 0.9928 f1_score: 0.6133 f2_score: 0.7958
69 instances of class C

Epoch 244/300
1009/1009 [==============================] - 694s 688ms/step - loss: 0.1387 - regression_loss: 0.1196 - classification_loss: 0.0191 - val_loss: 0.0564 - val_regression_loss: 0.2102 - val_classification_loss: 0.1022
Running network: 100% (524 of 524) |#####| Elapsed Time: 0:00:49 Time:  0:00:49
Parsing annotations: 100% (524 of 524) |#| Elapsed Time: 0:00:00 Time:  0:00:00
99 instances of class Animal with average precision: 0.7443 TP: 87 FP: 643 precision: 0.1192 recall: 0.8788 f1_score: 0.2099 f2_score: 0.3863
65 instances of class Basural-Escombro-MConstrucción with average precision: 0.2489 TP: 46 FP: 1033 precision: 0.0426 recall: 0.7077 f1_score: 0.0804 f2_score: 0.1718
0 instances of class Bus with average precision: 0.0000 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_score: 0.0000 f2_score: 0.0000
139 instances of class Camión with average precision: 0.9782 TP: 138 FP: 173 precision: 0.4437 recall: 0.9928 f1_score: 0.6133 f2_score: 0.7958
69 instances of class C

Epoch 247/300
1009/1009 [==============================] - 708s 702ms/step - loss: 0.1387 - regression_loss: 0.1186 - classification_loss: 0.0202 - val_loss: 0.0548 - val_regression_loss: 0.2097 - val_classification_loss: 0.1018
Running network: 100% (524 of 524) |#####| Elapsed Time: 0:00:49 Time:  0:00:49
Parsing annotations: 100% (524 of 524) |#| Elapsed Time: 0:00:00 Time:  0:00:00
99 instances of class Animal with average precision: 0.7437 TP: 87 FP: 642 precision: 0.1193 recall: 0.8788 f1_score: 0.2101 f2_score: 0.3867
65 instances of class Basural-Escombro-MConstrucción with average precision: 0.2474 TP: 46 FP: 1038 precision: 0.0424 recall: 0.7077 f1_score: 0.0801 f2_score: 0.1711
0 instances of class Bus with average precision: 0.0000 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_score: 0.0000 f2_score: 0.0000
139 instances of class Camión with average precision: 0.9782 TP: 138 FP: 174 precision: 0.4423 recall: 0.9928 f1_score: 0.6120 f2_score: 0.7949
69 instances of class C

## New training:
    - Top layers only
    - gamma = 2.5
    - alpha = 0.5

In [16]:
import pandas as pd
import math

BATCH_SIZE = 8

# compute number of annotations to compute number of steps per epoch
df = pd.read_csv(TRAIN_ANNOTATIONS, header=None, names=['img_path', 'x1', 'y1', 'x2', 'y2', 'class'])

countAnn = len(df)-1
countImg = df['img_path'].nunique()
no_steps = math.ceil(countImg/BATCH_SIZE)

print("Count of images: {}".format(countImg))
print("Count of annotations: {}".format(countAnn))
print("Number of steps per epoch: {}".format(no_steps))

Count of images: 4035
Count of annotations: 6412
Number of steps per epoch: 505


# Train model

In [17]:
CONFIG_FILE = os.path.join(TRAINING_BASE_DIR, 'config.ini')
TENSORBOARD_LOGS_DIR = os.path.join(TRAINING_BASE_DIR, 'logs_iter3_change_gamma_alpha_more_false_positives')
SNAPSHOTS_DIR = os.path.join(TRAINING_BASE_DIR, 'snapshots')

epoch = 191
last_model = os.path.join(SNAPSHOTS_DIR, 'resnet50_csv_{}.h5'.format(epoch))

SNAPSHOTS_DIR = os.path.join(TRAINING_BASE_DIR, 'snapshots_iter3_change_gamma_alpha_more_false_positives')

In [18]:
TENSORBOARD_LOGS_DIR

'../trainings/v5/logs_iter3_change_gamma_alpha_more_false_positives'

### FCNs training (regression, classification) with freezed backbone  

First, train the model freezing the backbone. The backbone weights will be the weights from the previous model train before.
Set gamma with high value (>2.0) to weight the loss of well-classified classes down, forcing the model to learn on harder (in this case: less examples) classes.

In [19]:
!python ../../keras-retinanet/keras_retinanet/bin/train.py \
--config {CONFIG_FILE} \
--freeze-backbone \
--random-transform \
--weights {last_model} \
--weighted-average \
--batch-size {BATCH_SIZE} \
--steps {no_steps} \
--epochs 100 \
--no-resize \
--image-min-side 1000 \
--image-max-side 1000 \
--compute-val-loss \
--multiprocessing \
--workers 2 \
--tensorboard-dir {TENSORBOARD_LOGS_DIR} \
--snapshot-path {SNAPSHOTS_DIR} \
--validation-freq 5 \
--gamma 2.5 \
--alpha 0.5 \
--class-names '/home/aikauel/enap/aerialnet_project/datasets/data/v5/CLASSES_REPORT_NAMES' \
csv {TRAIN_ANNOTATIONS} {CLASSES_FILE} \
--val-annotations {VAL_ANNOTATIONS}

Using TensorFlow backend.
classNames: ['Animal', 'Basural-Escombro-MConstrucción', 'Bus', 'Camión', 'Chasis', 'Cilindro', 'Estructura', 'GHorquilla', 'Juegos', 'Maquinaria', 'PalletCaja', 'Persona', 'Pickup', 'Piscina', 'Poste', 'SAdvertencia', 'Tractor', 'Troncos', 'Tuberia', 'Vehículo']
Creating model, this may take a second...
2020-11-10 16:10:00.310667: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-11-10 16:10:00.339192: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-11-10 16:10:00.339552: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.755GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2020-11-10 16:10:00.3

tracking <tf.Variable 'Variable:0' shape=(27, 4) dtype=float32, numpy=
array([[-28.992   ,  -7.248   ,  28.992   ,   7.248   ],
       [-46.016   , -11.504   ,  46.016   ,  11.504   ],
       [-73.216   , -18.304   ,  73.216   ,  18.304   ],
       [-22.778267,  -9.225198,  22.778267,   9.225198],
       [-36.153584, -14.642201,  36.153584,  14.642201],
       [-57.523926, -23.29719 ,  57.523926,  23.29719 ],
       [-19.353823, -10.857494,  19.353823,  10.857494],
       [-30.718317, -17.232977,  30.718317,  17.232977],
       [-48.87588 , -27.419369,  48.87588 ,  27.419369],
       [-16.716267, -12.570632,  16.716267,  12.570632],
       [-26.531998, -19.95206 ,  26.531998,  19.95206 ],
       [-42.21503 , -31.745703,  42.21503 ,  31.745703],
       [-14.496   , -14.496   ,  14.496   ,  14.496   ],
       [-23.008   , -23.008   ,  23.008   ,  23.008   ],
       [-36.608   , -36.608   ,  36.608   ,  36.608   ],
       [-12.564904, -16.723886,  12.564904,  16.723886],
       [-19.94297

2020-11-10 16:10:02.177380: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1408] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI_ERROR_INSUFFICIENT_PRIVILEGES
2020-11-10 16:10:02.179006: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1447] function cupti_interface_->ActivityRegisterCallbacks( AllocCuptiActivityBuffer, FreeCuptiActivityBuffer)failed with error CUPTI_ERROR_INSUFFICIENT_PRIVILEGES
2020-11-10 16:10:02.179062: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1430] function cupti_interface_->EnableCallback( 0 , subscriber_, CUPTI_CB_DOMAIN_DRIVER_API, cbid)failed with error CUPTI_ERROR_INVALID_PARAMETER
Epoch 1/100
2020-11-10 16:10:12.744039: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2020-11-10 16:10:14.147863: W tensorflow/stream_executor/gpu/asm_compiler.cc:81] Running ptxas --version returned 256
2020-11-10 16:1

Epoch 3/100
505/505 [==============================] - 532s 1s/step - loss: 0.2112 - regression_loss: 0.1408 - classification_loss: 0.0703
Running network: 100% (524 of 524) |#####| Elapsed Time: 0:00:50 Time:  0:00:50
Parsing annotations: 100% (524 of 524) |#| Elapsed Time: 0:00:00 Time:  0:00:00
99 instances of class Animal with average precision: 0.7401 TP: 88 FP: 881 precision: 0.0908 recall: 0.8889 f1_score: 0.1648 f2_score: 0.3223
65 instances of class Basural-Escombro-MConstrucción with average precision: 0.2227 TP: 46 FP: 998 precision: 0.0441 recall: 0.7077 f1_score: 0.0830 f2_score: 0.1764
0 instances of class Bus with average precision: 0.0000 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_score: 0.0000 f2_score: 0.0000
139 instances of class Camión with average precision: 0.9823 TP: 139 FP: 334 precision: 0.2939 recall: 1.0000 f1_score: 0.4542 f2_score: 0.6754
69 instances of class Chasis with average precision: 0.9708 TP: 69 FP: 544 precision: 0.1126 recall: 1.0000 f1_sco

Epoch 6/100
505/505 [==============================] - 564s 1s/step - loss: 0.2055 - regression_loss: 0.1413 - classification_loss: 0.0642 - val_loss: 2.0234 - val_regression_loss: 0.2354 - val_classification_loss: 0.1521
Running network: 100% (524 of 524) |#####| Elapsed Time: 0:00:51 Time:  0:00:51
Parsing annotations: 100% (524 of 524) |#| Elapsed Time: 0:00:00 Time:  0:00:00
99 instances of class Animal with average precision: 0.7584 TP: 89 FP: 691 precision: 0.1141 recall: 0.8990 f1_score: 0.2025 f2_score: 0.3784
65 instances of class Basural-Escombro-MConstrucción with average precision: 0.2232 TP: 45 FP: 1037 precision: 0.0416 recall: 0.6923 f1_score: 0.0785 f2_score: 0.1677
0 instances of class Bus with average precision: 0.0000 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_score: 0.0000 f2_score: 0.0000
139 instances of class Camión with average precision: 0.9840 TP: 139 FP: 350 precision: 0.2843 recall: 1.0000 f1_score: 0.4427 f2_score: 0.6651
69 instances of class Chasis w

Epoch 9/100
505/505 [==============================] - 529s 1s/step - loss: 0.1953 - regression_loss: 0.1339 - classification_loss: 0.0614 - val_loss: 2.0234 - val_regression_loss: 0.2354 - val_classification_loss: 0.1521
Running network: 100% (524 of 524) |#####| Elapsed Time: 0:00:50 Time:  0:00:50
Parsing annotations: 100% (524 of 524) |#| Elapsed Time: 0:00:00 Time:  0:00:00
99 instances of class Animal with average precision: 0.7580 TP: 89 FP: 678 precision: 0.1160 recall: 0.8990 f1_score: 0.2055 f2_score: 0.3826
65 instances of class Basural-Escombro-MConstrucción with average precision: 0.2230 TP: 45 FP: 1002 precision: 0.0430 recall: 0.6923 f1_score: 0.0809 f2_score: 0.1721
0 instances of class Bus with average precision: 0.0000 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_score: 0.0000 f2_score: 0.0000
139 instances of class Camión with average precision: 0.9847 TP: 139 FP: 334 precision: 0.2939 recall: 1.0000 f1_score: 0.4542 f2_score: 0.6754
69 instances of class Chasis w

Epoch 12/100
505/505 [==============================] - 540s 1s/step - loss: 0.1970 - regression_loss: 0.1347 - classification_loss: 0.0622 - val_loss: 2.0571 - val_regression_loss: 0.2355 - val_classification_loss: 0.1544
Running network: 100% (524 of 524) |#####| Elapsed Time: 0:00:51 Time:  0:00:51
Parsing annotations: 100% (524 of 524) |#| Elapsed Time: 0:00:00 Time:  0:00:00
99 instances of class Animal with average precision: 0.7419 TP: 88 FP: 675 precision: 0.1153 recall: 0.8889 f1_score: 0.2042 f2_score: 0.3796
65 instances of class Basural-Escombro-MConstrucción with average precision: 0.2394 TP: 45 FP: 1058 precision: 0.0408 recall: 0.6923 f1_score: 0.0771 f2_score: 0.1651
0 instances of class Bus with average precision: 0.0000 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_score: 0.0000 f2_score: 0.0000
139 instances of class Camión with average precision: 0.9840 TP: 139 FP: 348 precision: 0.2854 recall: 1.0000 f1_score: 0.4441 f2_score: 0.6663
69 instances of class Chasis 

Epoch 15/100
505/505 [==============================] - 647s 1s/step - loss: 0.1960 - regression_loss: 0.1344 - classification_loss: 0.0616 - val_loss: 2.0598 - val_regression_loss: 0.2317 - val_classification_loss: 0.1529
Running network: 100% (524 of 524) |#####| Elapsed Time: 0:00:49 Time:  0:00:49
Parsing annotations: 100% (524 of 524) |#| Elapsed Time: 0:00:00 Time:  0:00:00
99 instances of class Animal with average precision: 0.7407 TP: 88 FP: 681 precision: 0.1144 recall: 0.8889 f1_score: 0.2028 f2_score: 0.3777
65 instances of class Basural-Escombro-MConstrucción with average precision: 0.2427 TP: 45 FP: 1048 precision: 0.0412 recall: 0.6923 f1_score: 0.0777 f2_score: 0.1663
0 instances of class Bus with average precision: 0.0000 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_score: 0.0000 f2_score: 0.0000
139 instances of class Camión with average precision: 0.9839 TP: 139 FP: 331 precision: 0.2957 recall: 1.0000 f1_score: 0.4565 f2_score: 0.6774
69 instances of class Chasis 

#### Full model training

In [25]:
BATCH_SIZE = 4

# compute number of annotations to compute number of steps per epoch
df = pd.read_csv(TRAIN_ANNOTATIONS, header=None, names=['img_path', 'x1', 'y1', 'x2', 'y2', 'class'])

countAnn = len(df)-1
countImg = df['img_path'].nunique()
no_steps = math.ceil(countImg/BATCH_SIZE)

print("Count of images: {}".format(countImg))
print("Count of annotations: {}".format(countAnn))
print("Number of steps per epoch: {}".format(no_steps))

Count of images: 4035
Count of annotations: 6412
Number of steps per epoch: 1009


In [26]:
epoch = 15
last_model = os.path.join(SNAPSHOTS_DIR, 'resnet50_csv_{}.h5'.format(epoch))

!python ../../keras-retinanet/keras_retinanet/bin/train.py \
--config {CONFIG_FILE} \
--random-transform \
--weights {last_model} \
--initial-epoch {epoch} \
--weighted-average \
--batch-size {BATCH_SIZE} \
--steps {no_steps} \
--epochs 200 \
--no-resize \
--image-min-side 1000 \
--image-max-side 1000 \
--compute-val-loss \
--multiprocessing \
--workers 2 \
--tensorboard-dir {TENSORBOARD_LOGS_DIR} \
--snapshot-path {SNAPSHOTS_DIR} \
--validation-freq 5 \
--lr 1e-5 \
--reduce-lr-patience 5 \
--reduce-lr-factor 0.1 \
--gamma 3.0 \
--class-names '/home/aikauel/enap/aerialnet_project/datasets/data/v5/CLASSES_REPORT_NAMES' \
csv {TRAIN_ANNOTATIONS} {CLASSES_FILE} \
--val-annotations {VAL_ANNOTATIONS}

Using TensorFlow backend.
classNames: ['Animal', 'Basural-Escombro-MConstrucción', 'Bus', 'Camión', 'Chasis', 'Cilindro', 'Estructura', 'GHorquilla', 'Juegos', 'Maquinaria', 'PalletCaja', 'Persona', 'Pickup', 'Piscina', 'Poste', 'SAdvertencia', 'Tractor', 'Troncos', 'Tuberia', 'Vehículo']
Creating model, this may take a second...
2020-11-10 19:06:44.810069: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-11-10 19:06:44.839793: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-11-10 19:06:44.840121: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.755GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2020-11-10 19:06:44.8

tracking <tf.Variable 'Variable:0' shape=(27, 4) dtype=float32, numpy=
array([[ -57.984   ,  -14.496   ,   57.984   ,   14.496   ],
       [ -92.032   ,  -23.008   ,   92.032   ,   23.008   ],
       [-146.432   ,  -36.608   ,  146.432   ,   36.608   ],
       [ -45.556534,  -18.450396,   45.556534,   18.450396],
       [ -72.30717 ,  -29.284403,   72.30717 ,   29.284403],
       [-115.04785 ,  -46.59438 ,  115.04785 ,   46.59438 ],
       [ -38.707645,  -21.714989,   38.707645,   21.714989],
       [ -61.436634,  -34.465954,   61.436634,   34.465954],
       [ -97.75176 ,  -54.838737,   97.75176 ,   54.838737],
       [ -33.432533,  -25.141264,   33.432533,   25.141264],
       [ -53.063995,  -39.90412 ,   53.063995,   39.90412 ],
       [ -84.43006 ,  -63.491405,   84.43006 ,   63.491405],
       [ -28.992   ,  -28.992   ,   28.992   ,   28.992   ],
       [ -46.016   ,  -46.016   ,   46.016   ,   46.016   ],
       [ -73.216   ,  -73.216   ,   73.216   ,   73.216   ],
       [ -25.1

2020-11-10 19:06:46.497061: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcupti.so.10.1
2020-11-10 19:06:46.599291: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1408] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI_ERROR_INSUFFICIENT_PRIVILEGES
2020-11-10 19:06:46.603633: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1447] function cupti_interface_->ActivityRegisterCallbacks( AllocCuptiActivityBuffer, FreeCuptiActivityBuffer)failed with error CUPTI_ERROR_INSUFFICIENT_PRIVILEGES
2020-11-10 19:06:46.603751: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1430] function cupti_interface_->EnableCallback( 0 , subscriber_, CUPTI_CB_DOMAIN_DRIVER_API, cbid)failed with error CUPTI_ERROR_INVALID_PARAMETER
Epoch 16/200
2020-11-10 19:07:02.956959: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library l

Epoch 17/200
1009/1009 [==============================] - 691s 684ms/step - loss: 0.1827 - regression_loss: 0.1441 - classification_loss: 0.0387
Running network: 100% (524 of 524) |#####| Elapsed Time: 0:00:49 Time:  0:00:49
Parsing annotations: 100% (524 of 524) |#| Elapsed Time: 0:00:00 Time:  0:00:00
99 instances of class Animal with average precision: 0.7408 TP: 87 FP: 797 precision: 0.0984 recall: 0.8788 f1_score: 0.1770 f2_score: 0.3398
65 instances of class Basural-Escombro-MConstrucción with average precision: 0.1872 TP: 42 FP: 1003 precision: 0.0402 recall: 0.6462 f1_score: 0.0757 f2_score: 0.1609
0 instances of class Bus with average precision: 0.0000 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_score: 0.0000 f2_score: 0.0000
139 instances of class Camión with average precision: 0.9727 TP: 138 FP: 316 precision: 0.3040 recall: 0.9928 f1_score: 0.4654 f2_score: 0.6832
69 instances of class Chasis with average precision: 0.9685 TP: 69 FP: 377 precision: 0.1547 recall: 1.0000

Epoch 20/200
1009/1009 [==============================] - 737s 731ms/step - loss: 0.1803 - regression_loss: 0.1442 - classification_loss: 0.0361 - val_loss: 0.0933 - val_regression_loss: 0.2295 - val_classification_loss: 0.0839
Running network: 100% (524 of 524) |#####| Elapsed Time: 0:00:48 Time:  0:00:48
Parsing annotations: 100% (524 of 524) |#| Elapsed Time: 0:00:00 Time:  0:00:00
99 instances of class Animal with average precision: 0.6921 TP: 85 FP: 632 precision: 0.1185 recall: 0.8586 f1_score: 0.2083 f2_score: 0.3819
65 instances of class Basural-Escombro-MConstrucción with average precision: 0.1986 TP: 43 FP: 943 precision: 0.0436 recall: 0.6615 f1_score: 0.0818 f2_score: 0.1726
0 instances of class Bus with average precision: 0.0000 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_score: 0.0000 f2_score: 0.0000
139 instances of class Camión with average precision: 0.9846 TP: 139 FP: 255 precision: 0.3528 recall: 1.0000 f1_score: 0.5216 f2_score: 0.7316
69 instances of class Cha

...

Epoch 35/200
1009/1009 [==============================] - 742s 735ms/step - loss: 0.1495 - regression_loss: 0.1228 - classification_loss: 0.0267 - val_loss: 0.0711 - val_regression_loss: 0.2172 - val_classification_loss: 0.0894
Running network: 100% (524 of 524) |#####| Elapsed Time: 0:00:49 Time:  0:00:49
Parsing annotations: 100% (524 of 524) |#| Elapsed Time: 0:00:00 Time:  0:00:00
99 instances of class Animal with average precision: 0.6632 TP: 79 FP: 632 precision: 0.1111 recall: 0.7980 f1_score: 0.1951 f2_score: 0.3568
65 instances of class Basural-Escombro-MConstrucción with average precision: 0.2316 TP: 45 FP: 1038 precision: 0.0416 recall: 0.6923 f1_score: 0.0784 f2_score: 0.1675
0 instances of class Bus with average precision: 0.0000 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_score: 0.0000 f2_score: 0.0000
139 instances of class Camión with average precision: 0.9860 TP: 138 FP: 244 precision: 0.3613 recall: 0.9928 f1_score: 0.5298 f2_score: 0.7356
69 instances of class Ch

Epoch 38/200
1009/1009 [==============================] - 684s 678ms/step - loss: 0.1502 - regression_loss: 0.1240 - classification_loss: 0.0262 - val_loss: 0.0711 - val_regression_loss: 0.2172 - val_classification_loss: 0.0894
Running network: 100% (524 of 524) |#####| Elapsed Time: 0:00:49 Time:  0:00:49
Parsing annotations: 100% (524 of 524) |#| Elapsed Time: 0:00:00 Time:  0:00:00
99 instances of class Animal with average precision: 0.6982 TP: 84 FP: 657 precision: 0.1134 recall: 0.8485 f1_score: 0.2000 f2_score: 0.3694
65 instances of class Basural-Escombro-MConstrucción with average precision: 0.2022 TP: 43 FP: 1057 precision: 0.0391 recall: 0.6615 f1_score: 0.0738 f2_score: 0.1581
0 instances of class Bus with average precision: 0.0000 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_score: 0.0000 f2_score: 0.0000
139 instances of class Camión with average precision: 0.9831 TP: 138 FP: 206 precision: 0.4012 recall: 0.9928 f1_score: 0.5714 f2_score: 0.7667
69 instances of class Ch


Epoch 00040: ReduceLROnPlateau reducing learning rate to 9.999999974752428e-08.
Epoch 41/200
1009/1009 [==============================] - 682s 676ms/step - loss: 0.1485 - regression_loss: 0.1238 - classification_loss: 0.0247 - val_loss: 0.0702 - val_regression_loss: 0.2148 - val_classification_loss: 0.0943
Running network: 100% (524 of 524) |#####| Elapsed Time: 0:00:49 Time:  0:00:49
Parsing annotations: 100% (524 of 524) |#| Elapsed Time: 0:00:00 Time:  0:00:00
99 instances of class Animal with average precision: 0.6771 TP: 80 FP: 662 precision: 0.1078 recall: 0.8081 f1_score: 0.1902 f2_score: 0.3515
65 instances of class Basural-Escombro-MConstrucción with average precision: 0.2182 TP: 41 FP: 1037 precision: 0.0380 recall: 0.6308 f1_score: 0.0717 f2_score: 0.1532
0 instances of class Bus with average precision: 0.0000 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_score: 0.0000 f2_score: 0.0000
139 instances of class Camión with average precision: 0.9826 TP: 138 FP: 217 precision:

Epoch 44/200
1009/1009 [==============================] - 684s 678ms/step - loss: 0.1483 - regression_loss: 0.1223 - classification_loss: 0.0260 - val_loss: 0.0702 - val_regression_loss: 0.2148 - val_classification_loss: 0.0943
Running network: 100% (524 of 524) |#####| Elapsed Time: 0:00:49 Time:  0:00:49
Parsing annotations: 100% (524 of 524) |#| Elapsed Time: 0:00:00 Time:  0:00:00
99 instances of class Animal with average precision: 0.6672 TP: 80 FP: 664 precision: 0.1075 recall: 0.8081 f1_score: 0.1898 f2_score: 0.3509
65 instances of class Basural-Escombro-MConstrucción with average precision: 0.2075 TP: 42 FP: 1043 precision: 0.0387 recall: 0.6462 f1_score: 0.0730 f2_score: 0.1561
0 instances of class Bus with average precision: 0.0000 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_score: 0.0000 f2_score: 0.0000
139 instances of class Camión with average precision: 0.9839 TP: 138 FP: 222 precision: 0.3833 recall: 0.9928 f1_score: 0.5531 f2_score: 0.7533
69 instances of class Ch

Epoch 00046: saving model to ../trainings/v5/snapshots_iter3_change_gamma_alpha_more_false_positives/resnet50_csv_46.h5
Epoch 47/200
1009/1009 [==============================] - 684s 678ms/step - loss: 0.1467 - regression_loss: 0.1208 - classification_loss: 0.0259 - val_loss: 0.0601 - val_regression_loss: 0.2103 - val_classification_loss: 0.0947
Running network: 100% (524 of 524) |#####| Elapsed Time: 0:00:49 Time:  0:00:49
Parsing annotations: 100% (524 of 524) |#| Elapsed Time: 0:00:00 Time:  0:00:00
99 instances of class Animal with average precision: 0.6787 TP: 81 FP: 662 precision: 0.1090 recall: 0.8182 f1_score: 0.1924 f2_score: 0.3556
65 instances of class Basural-Escombro-MConstrucción with average precision: 0.2131 TP: 42 FP: 1034 precision: 0.0390 recall: 0.6462 f1_score: 0.0736 f2_score: 0.1572
0 instances of class Bus with average precision: 0.0000 TP: 0 FP: 0 precision: 0.0000 recall: 0.0000 f1_score: 0.0000 f2_score: 0.0000
139 instances of class Camión with average preci

## Test model
#### Se procesan las imágenes del dataset de testing

In [45]:
# Convert training model to inference model
# path to trained weights
epoch = 228
last_model = os.path.join(SNAPSHOTS_DIR, 'resnet50_csv_{}.h5'.format(epoch))
inference_model = last_model.replace('snapshots', 'inference')

# convert trained model to inference model to be used for generating predictions
!python ../../keras-retinanet/keras_retinanet/bin/convert_model.py {last_model} {inference_model} \
--config {CONFIG_FILE}

Using TensorFlow backend.
2020-11-11 14:56:19.085009: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-11-11 14:56:19.117942: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-11-11 14:56:19.118252: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.755GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2020-11-11 14:56:19.118374: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-11-11 14:56:19.119450: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2020-11-11 14:56:19.120484: I tensorf

tracking <tf.Variable 'Variable:0' shape=(27, 4) dtype=float32, numpy=
array([[-231.936   ,  -57.984   ,  231.936   ,   57.984   ],
       [-368.128   ,  -92.032   ,  368.128   ,   92.032   ],
       [-585.728   , -146.432   ,  585.728   ,  146.432   ],
       [-182.22614 ,  -73.80158 ,  182.22614 ,   73.80158 ],
       [-289.22867 , -117.13761 ,  289.22867 ,  117.13761 ],
       [-460.1914  , -186.37752 ,  460.1914  ,  186.37752 ],
       [-154.83058 ,  -86.859955,  154.83058 ,   86.859955],
       [-245.74654 , -137.86382 ,  245.74654 ,  137.86382 ],
       [-391.00705 , -219.35495 ,  391.00705 ,  219.35495 ],
       [-133.73013 , -100.565056,  133.73013 ,  100.565056],
       [-212.25598 , -159.61649 ,  212.25598 ,  159.61649 ],
       [-337.72025 , -253.96562 ,  337.72025 ,  253.96562 ],
       [-115.968   , -115.968   ,  115.968   ,  115.968   ],
       [-184.064   , -184.064   ,  184.064   ,  184.064   ],
       [-292.864   , -292.864   ,  292.864   ,  292.864   ],
       [-100.5

In [46]:
SNAPSHOTS_DIR

'../trainings/v5/snapshots'

#### Set paths

In [56]:
test_images_path = '/home/aikauel/enap/aerialnet_project/datasets/official_37_test_images_ori/'
#test_images_path = '/home/aikauel/enap/data/vitto/'
test_output_path = os.path.join(TRAINING_BASE_DIR, 
                                'official_37_output_epoch{}_threshold53'.format(epoch))

In [57]:
THRESHOLD = 0.53
test_images_path = '/home/aikauel/enap/data/test_images/'
test_output_path = os.path.join(TRAINING_BASE_DIR, 
                                'data_test_images_output_epoch{}_threshold_{}'.format(epoch, THRESHOLD))

In [58]:
test_output_path
inference_model

'../trainings/v5/inference/resnet50_csv_228.h5'

In [59]:
!python ../../aerialnet_project/shared_utils/image_inference_write.py \
--input_dir {test_images_path} \
--model {inference_model} \
--output_dir {test_output_path} \
--labels {CLASSES_FILE} \
--threshold 0.53 \
--no_csv True

Using TensorFlow backend.
/home/aikauel/enap/data/test_images/
LABELS: {0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9', 10: '10', 11: '11', 12: '12', 13: '13', 14: '14', 15: '15', 16: '16', 17: '17', 18: '18', 19: '19'}
2020-11-11 17:00:08.361720: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-11-11 17:00:08.391291: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-11-11 17:00:08.391635: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.755GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2020-11-11 17:00:08.391768: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfu

/home/aikauel/anaconda3/envs/enap/lib/python3.6/site-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
no csv: /home/aikauel/enap/data/test_images/
[INFO] predicting image 1 of 49: 6425959702.jpg
2020-11-11 17:00:13.558287: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2020-11-11 17:00:14.260266: W tensorflow/stream_executor/gpu/asm_compiler.cc:81] Running ptxas --version returned 256
2020-11-11 17:00:14.320052: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2020-11-11 17:00:16.978384: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully 

[INFO] predicting image 38 of 49: 2020-01-22_16-03-24-4817751-DJI_0158.jpg
processing time:  0.6213991641998291
../trainings/v5/data_test_images_output_epoch228_threshold_0.53/2020-01-22_16-03-24-4817751-DJI_0158.jpg
[INFO] predicting image 39 of 49: 1075bb6045417762.JPG
processing time:  0.625708818435669
../trainings/v5/data_test_images_output_epoch228_threshold_0.53/1075bb6045417762.JPG
[INFO] predicting image 40 of 49: 475b77148273e8d.JPG
processing time:  0.6181647777557373
../trainings/v5/data_test_images_output_epoch228_threshold_0.53/475b77148273e8d.JPG
[INFO] predicting image 41 of 49: 10000185c338565a289d.JPG
processing time:  0.6201736927032471
../trainings/v5/data_test_images_output_epoch228_threshold_0.53/10000185c338565a289d.JPG
[INFO] predicting image 42 of 49: 10000055c1d16b33440e.JPG
processing time:  0.6235079765319824
../trainings/v5/data_test_images_output_epoch228_threshold_0.53/10000055c1d16b33440e.JPG
[INFO] predicting image 43 of 49: 115b4f3d4d2d7ca.JPG
processi

## Resultados de testing:
 #### Maquinaria: FP=1, TP=5, FN=5

# DEBUG

If the model is not learning or has low accuracy, check the dataset and the anchors of the model to determine if the problem lies in a mismatch between the dataset and the anchors.

In [47]:
!python ../../keras-retinanet/keras_retinanet/bin/debug.py \
--config {CONFIG_FILE} \
--show-annotations \
csv {TRAIN_ANNOTATIONS} {CLASSES_FILE}

Using TensorFlow backend.
Qt: Session management error: Could not open network socket
^C


In [84]:
%pwd

'/home/aikauel/enap'

# Evaluate model

Evaluate model with test dataset (test.csv)

In [37]:
# First, create inference model
# path to trained weights
epoch = 97
last_model = os.path.join(SNAPSHOTS_DIR, 'resnet50_csv_{}.h5'.format(epoch))
inference_model = last_model.replace('snapshots', 'inference')

# convert trained model to inference model to be used for generating predictions
inference_model = last_model.replace('snapshots/', 'inference/')
!python ../../keras-retinanet/keras_retinanet/bin/convert_model.py {last_model} {inference_model} \
--config {CONFIG_FILE}

Using TensorFlow backend.
2020-11-04 10:05:59.377068: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-11-04 10:05:59.414357: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-11-04 10:05:59.415009: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.755GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2020-11-04 10:05:59.415778: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-11-04 10:05:59.420140: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2020-11-04 10:05:59.423024: I tensorf

tracking <tf.Variable 'Variable:0' shape=(27, 4) dtype=float32, numpy=
array([[-231.936   ,  -57.984   ,  231.936   ,   57.984   ],
       [-368.128   ,  -92.032   ,  368.128   ,   92.032   ],
       [-585.728   , -146.432   ,  585.728   ,  146.432   ],
       [-182.22614 ,  -73.80158 ,  182.22614 ,   73.80158 ],
       [-289.22867 , -117.13761 ,  289.22867 ,  117.13761 ],
       [-460.1914  , -186.37752 ,  460.1914  ,  186.37752 ],
       [-154.83058 ,  -86.859955,  154.83058 ,   86.859955],
       [-245.74654 , -137.86382 ,  245.74654 ,  137.86382 ],
       [-391.00705 , -219.35495 ,  391.00705 ,  219.35495 ],
       [-133.73013 , -100.565056,  133.73013 ,  100.565056],
       [-212.25598 , -159.61649 ,  212.25598 ,  159.61649 ],
       [-337.72025 , -253.96562 ,  337.72025 ,  253.96562 ],
       [-115.968   , -115.968   ,  115.968   ,  115.968   ],
       [-184.064   , -184.064   ,  184.064   ,  184.064   ],
       [-292.864   , -292.864   ,  292.864   ,  292.864   ],
       [-100.5

In [63]:
#TEST_ANNOTATIONS = os.path.join(DATASET_VERSION_DIR, 'test.csv')
TEST_ANNOTATIONS = os.path.join(DATASET_VERSION_DIR, 'test.csv')

In [131]:
#output_images_path = os.path.join(TRAINING_BASE_DIR, 'test_output')

# evaluate model on test.csv
!python ../../keras-retinanet/keras_retinanet/bin/evaluate.py \
--config {CONFIG_FILE} \
--score-threshold 0.35 \
--iou-threshold 0.35 \
--no-resize \
--max-detections 300 \
csv {TEST_ANNOTATIONS} \
{CLASSES_FILE} \
{inference_model}

Using TensorFlow backend.
Loading model, this may take a second...
2020-09-16 22:21:20.231116: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-09-16 22:21:20.259592: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-09-16 22:21:20.259910: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.755GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2020-09-16 22:21:20.260052: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-09-16 22:21:20.261005: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so

/home/aikauel/anaconda3/envs/enap/lib/python3.6/site-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
Running network: N/A% (0 of 612) |       | Elapsed Time: 0:00:00 ETA:  --:--:--2020-09-16 22:21:23.278742: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2020-09-16 22:21:24.023666: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
Running network: 100% (612 of 612) |#####| Elapsed Time: 0:00:53 Time:  0:00:53
Parsing annotations: 100% (612 of 612) |#| Elapsed Time: 0:00:00 Time:  0:00:00
218 instances of class 0 with average precision: 0.9307 TP: 206 FP: 56 precision: 0.7863 recall: 0.9450 f1_score: 0.8583 f2_score: 0.9083
16 instances of class 1 with average precision: 0.4375 TP: 7 FP: 0 preci

## Test model
#### Test model on images without annotations (unseen in train-val-test)

In [39]:
# Convert training model to inference model
# path to trained weights
epoch = 97
last_model = os.path.join(SNAPSHOTS_DIR, 'resnet50_csv_{}.h5'.format(epoch))
inference_model = last_model.replace('snapshots', 'inference')

# convert trained model to inference model to be used for generating predictions
inference_model = last_model.replace('snapshots/', 'inference/')
!python ../../keras-retinanet/keras_retinanet/bin/convert_model.py {last_model} {inference_model} \
--config {CONFIG_FILE}

Using TensorFlow backend.
2020-11-04 10:12:05.457707: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-11-04 10:12:05.485863: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-11-04 10:12:05.486184: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.755GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2020-11-04 10:12:05.486331: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-11-04 10:12:05.487326: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2020-11-04 10:12:05.488254: I tensorf

tracking <tf.Variable 'Variable:0' shape=(27, 4) dtype=float32, numpy=
array([[-115.968   ,  -28.992   ,  115.968   ,   28.992   ],
       [-184.064   ,  -46.016   ,  184.064   ,   46.016   ],
       [-292.864   ,  -73.216   ,  292.864   ,   73.216   ],
       [ -91.11307 ,  -36.90079 ,   91.11307 ,   36.90079 ],
       [-144.61433 ,  -58.568806,  144.61433 ,   58.568806],
       [-230.0957  ,  -93.18876 ,  230.0957  ,   93.18876 ],
       [ -77.41529 ,  -43.429977,   77.41529 ,   43.429977],
       [-122.87327 ,  -68.93191 ,  122.87327 ,   68.93191 ],
       [-195.50352 , -109.677475,  195.50352 ,  109.677475],
       [ -66.86507 ,  -50.282528,   66.86507 ,   50.282528],
       [-106.12799 ,  -79.80824 ,  106.12799 ,   79.80824 ],
       [-168.86012 , -126.98281 ,  168.86012 ,  126.98281 ],
       [ -57.984   ,  -57.984   ,   57.984   ,   57.984   ],
       [ -92.032   ,  -92.032   ,   92.032   ,   92.032   ],
       [-146.432   , -146.432   ,  146.432   ,  146.432   ],
       [ -50.2

#### Set paths

In [40]:
test_images_path = '/home/aikauel/enap/aerialnet_project/datasets/official_37_test_images_ori/'
#test_images_path = '/home/aikauel/enap/data/vitto/'
test_output_path = os.path.join(TRAINING_BASE_DIR, 
                                'official_37_output_epoch{}_threshold53'.format(epoch))

In [41]:
test_output_path

'../trainings/v5/official_37_output_epoch97_threshold53'

In [44]:
!python ../../aerialnet_project/shared_utils/image_inference_write.py \
--input_dir {test_images_path} \
--model {inference_model} \
--output_dir {test_output_path} \
--labels {CLASSES_FILE} \
--threshold 0.53 \
--no_csv True

Using TensorFlow backend.
/home/aikauel/enap/aerialnet_project/datasets/official_37_test_images_ori/
LABELS: {0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9', 10: '10', 11: '11', 12: '12', 13: '13', 14: '14', 15: '15', 16: '16', 17: '17', 18: '18', 19: '19'}
Output dir already exist
2020-11-04 10:24:00.851501: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-11-04 10:24:00.895179: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-11-04 10:24:00.895707: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.755GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2020-11-04 10:24:00.895906: I tensorfl

/home/aikauel/anaconda3/envs/enap/lib/python3.6/site-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
no csv: /home/aikauel/enap/aerialnet_project/datasets/official_37_test_images_ori/
[INFO] predicting image 1 of 37: 2020-08-11_08:13:26-3872762-DJI_0282.jpg
2020-11-04 10:24:04.205793: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2020-11-04 10:24:04.845317: W tensorflow/stream_executor/gpu/asm_compiler.cc:81] Running ptxas --version returned 256
2020-11-04 10:24:04.892890: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2020-11-04 10:24:06.022271: I tensorflow/

# Create saved_model to load tfserving model

In [42]:
last_model = os.path.join(SNAPSHOTS_DIR, 'resnet50_csv_107.h5')

saved_model = os.path.dirname(last_model).replace('snapshots_iter2', 'saved_models')

In [43]:
print(last_model, saved_model)

../trainings/v3/snapshots_iter2/resnet50_csv_107.h5 ../trainings/v3/saved_models


In [44]:
!python ../shared_utils/convert_model.py {last_model} {saved_model} \
--config {CONFIG_FILE}

Using TensorFlow backend.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
2020-09-01 18:34:37.042366: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-09-01 18:34:37.074438: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-09-01 18:34:37.074740: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.755GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2020-09-01 18:34:37.074883: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-09-01 18:34:37.075773: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successf

[INFO] Model saved


## Visualization
#### Generate detections on test images - IMAGES

In [46]:
os.getcwd()

'/home/aikauel/enap'

In [13]:
# create output directory where you want to save images with bounding boxes
#!mkdir /content/data/output

# generate detections on images
!python ije_retinanet/image_inference_print.py \
-i 'data/test_images' \
-t 0.6 \
-m  {inference_model} \
-o data/output

python: can't open file 'ije_retinanet/image_inference_print.py': [Errno 2] No such file or directory
